In [ ]:
import os
import sys
sys.path.insert(0, '~/COAT') 

from utils import Logger, get_value_from_responce
from openai_utils import LLM_openai


import numpy as np
import pandas as pd
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.utils.GraphUtils import GraphUtils
from copy import  deepcopy
from scipy import stats
from sklearn.cluster import KMeans
from causallearn.utils.cit import CIT

# load groundtruth
meta = pd.read_excel('neuro_R_shoulder_impingement.xlsx')



In [2]:
data_interface = pd.read_csv("SimulatedData_SampleSize100.csv")
del data_interface['Unnamed: 0']
data_interface_name = pd.read_csv("id_name.txt", sep=':', header = None)
data_interface.columns = [nm.strip() for nm in data_interface_name.values[:,-1]]

data_interface_2 = pd.read_csv("neuro_addition_1000_0126.csv")

data_interface = pd.concat([data_interface, data_interface_2])


names = list(meta.columns[1:])
values = data_interface[names].values.astype(float)

data_interface

,DLI C1-C2,DLI C2-C3,DLI C3-C4,DLI C4-C5,DLI C5-C6,DLI C6-C7,DLI C7-C8,DLI C8-T1,DLI L1-L2,DLI L2-L3,...,Breast backache,Chest discomfort,L breast problems,R breast problems,Toracal dysfunction,Upper abdominal discomfort,Lateral abdominal discomfort,Abdominal discomfort,L lower abdominal discomfort,Lower abdominal discomfort
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
998,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
san_check_names = ['R shoulder impingement', 'R C4 Radiculopathy', 'R C5 Radiculopathy', 'R C6 Radiculopathy', 'DLI C3-C4', 'DLI C4-C5', 'DLI C5-C6']
san_check_values = data_interface[san_check_names].values.astype(float)


# Causal Discovery (FCI)
g, edges = fci(san_check_values.astype(float), alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=san_check_names)
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 801.71it/s]

digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="R C4 Radiculopathy"];
2 [label=X3];
2 [label="R C5 Radiculopathy"];
3 [label=X4];
3 [label="R C6 Radiculopathy"];
4 [label=X5];
4 [label="DLI C3-C4"];
5 [label=X6];
5 [label="DLI C4-C5"];
6 [label=X7];
6 [label="DLI C5-C6"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
5 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
6 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [12]:
target_name = 'R shoulder impingement'

In [13]:

# Causal Discovery (FCI)
g, edges = fci(values.astype(float), alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=names)
print(pdy.to_string())

Depth=0, working on node 3: 100%|██████████| 4/4 [00:00<00:00, 963.16it/s]

digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="R C4 Radiculopathy"];
2 [label=X3];
2 [label="R C5 Radiculopathy"];
3 [label=X4];
3 [label="R C6 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
}



# Utils

In [14]:
allowed_names = data_interface.columns[:80]

location_list = []
for a in 'ctls':
    for b in range(15):
        location_list.append(a+f'{b+1}')

In [15]:
def get_variables_with_data(intput_names):
    output_names = []
    for each_name in intput_names:
        each_name = ' ' + each_name.lower()

        if 'radi' in each_name:
            possible_vars = []
            for kl in location_list:
                if kl in each_name:
                    possible_vars += [v for v in allowed_names if kl in v.lower()]

            possible_vars = list(set(possible_vars))

            tmp = [v for v in possible_vars if 'Radiculopathy' in v]

            surround = [ '"', "'",] + list(' /\\[]().,;+-*=_`~!')

            contain_left = False 
            for kwd_ in ['l', 'left']:
                for add_l in surround:
                    for add_r in surround:
                        if add_l+kwd_+add_r in each_name:
                            contain_left = True

            contain_right = False 
            for kwd_ in ['r', 'right']:
                for add_l in surround:
                    for add_r in surround:
                        if add_l+kwd_+add_r in each_name:
                            contain_right = True

            if not (contain_left or contain_right):
                contain_left = True
                contain_right = True

            tmp_l, tmp_r = [], []
            if contain_left:
                tmp_l = [v for v in tmp if 'L ' in v]
            if contain_right:
                tmp_r = [v for v in tmp if 'R ' in v]
            output_names += tmp_l + tmp_r

        if 'dli' in each_name:
            possible_vars = []
            for kl in location_list:
                if kl in each_name:
                    if kl+'-' in each_name:
                        possible_vars += [v for v in allowed_names if kl+'-' in v.lower()]
                    if '-'+kl in each_name:
                        possible_vars += [v for v in allowed_names if '-'+kl in v.lower()]
                    if ' '+kl in each_name:
                        possible_vars += [v for v in allowed_names if ' '+kl in v.lower()]

            possible_vars = list(set(possible_vars))

            output_names += [v for v in possible_vars if 'DLI' in v]
        output_names = list(set(output_names))
    return output_names

In [16]:
# sorted_Neuro_RSI-3 symptom only.txt

input_factors = '''  
DLI C1-C2 (discoligamentous injury in the cervical spine)
L/R C4-C5 (radiculopathy in the cervical spine)
L/R L1-L2 (radiculopathy in the lumbar spine)
'''.split('\n')
get_variables_with_data(input_factors)

['L L1 Radiculopathy',
 'DLI C1-C2',
 'R C5 Radiculopathy',
 'L C4 Radiculopathy',
 'L C5 Radiculopathy',
 'R C4 Radiculopathy',
 'R L2 Radiculopathy',
 'R L1 Radiculopathy',
 'L L2 Radiculopathy']

In [17]:
get_variables_with_data(['DLI c1', 'DLI t3-t5', 'r radi c2', 'radi c4-c5'])

['DLI C1-C2',
 'R C5 Radiculopathy',
 'L C4 Radiculopathy',
 'L C5 Radiculopathy',
 'DLI T4-T5',
 'R C4 Radiculopathy',
 'R C2 Radiculopathy',
 'DLI T3-T4']

In [9]:
init_data = ''' 

### Group with 'R shoulder impingement'=0

----


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impingement, accompanied by non-specific problems with the right shoulder. The patient also reports issues in the lower body, including concerns with the left adductor tendon, unspecified lumbago, discomfort in the left hip joint, and pain in the right shin. Further symptoms involve trouble with the left knee, pain in the left hamstring, issues with the left toe, and pain located in the left rear thigh.

----


Patient ID: Neuro_00040
Age: 19
Gender: Male

Symptom Diagnosis:
The patient presents with a range of symptoms predominantly relating to the lower back and left lower limb. He reports lumbago, significant discomfort localized in the left hip joint, as well as issues involving the left ankle and footstool. The constellation of these symptoms suggests a potential underlying musculoskeletal or neurological condition focusing on the lower left extremity.

----


Patient ID: Neuro_00014
Age: 20
Gender: Male

Symptom Diagnosis Summary:
The patient presents with pain predominantly in the cervical and lumbosacral regions, manifested as neck pain, with specific complaints in the right neck area. Additionally, the patient reports issues with the right front axle, which could be descriptive of bodily or structural discomfort in that region. Musculoskeletal troubles are reported, including trouble in the left shoulder, both right and left shoulder problems, and right shoulder trouble, indicating discomfort and potential dysfunction in these areas. Upper extremity complaints are present as well, with the patient experiencing problems in the right medial elbow, right hand, and left wrist, as well as trouble with the left fingers. The patient also mentions left adductor tendon issues, alongside general lower back pain described as lumbago. Furthermore, pain is noted in the left thigh, left front knee, left shin, right big toe, and right heel.


### Group with 'R shoulder impingement'=1

----


Patient ID: Neuro_00087
Age: 27
Gender: Female

Symptom Diagnosis:
The patient presents with a myriad of complaints, primarily musculoskeletal in nature. There is an issue noted with right shoulder impingement and general problems in the same area, including unspecified trouble. The left shoulder too presents with difficulties, although it was reported that she can give a thumbs-up, suggesting some preserved motor function. Notably, there are also problems with the left hand alongside a backache and lumbago. Extending from the lumbar complaints, the individual has described pain in the right hamstring, heel issues on the same side, and pain located in the left rear thigh. Additionally, right achillodynia has been diagnosed.


----


Patient ID: Neuro_00012
Age: 13
Gender: Male

Symptom Diagnosis Summary:
The patient is exhibiting a cluster of symptoms that include problems with the left eye, a consistent headache concentrated at the forehead, and impingement with concurrent problems and trouble in the right shoulder. Additionally, the patient is experiencing lumbago, which is lower back pain, along with pain in both the left and right rear thighs. These symptoms may be associated with his neurological condition and should be further evaluated to determine their etiology and relation to one another.



----


Patient ID: Neuro_00070
Age: 18
Gender: Male

Symptom Diagnosis:
The patient presents with a range of right-sided upper extremity issues, including pain in the right neck, impingement in the right shoulder, trouble with both the right shoulder and right elbow, and wearing an armband on the right arm. Additionally, there are problems on the left side, with the patient able to give a thumbs-up, but reporting issues with the left wrist and left hand.



'''


In [18]:
# **What are the high-level factors behind the text that contribute to the allocation of groups?**
propose_prompt_with_data_in_middle = lambda example: f'''
You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Tasks: Factor Abstraction.

**What are the high-level factors that may cause the right shoulder impingement?**

Propose your candidate factors based on your observation and analysis on given data. **Only factors in following Templates are allowed**:
- Templates 1: **"DLI [location on the dermatome map]"**
- Templates 2: **"[L/R] Radiculopathy [location on the dermatome map]"**

Meaning of abbreviation:
- L: Left
- R: Right
- DLI: discoligamentous injury
- Radi: radiculopathy 
- C1, C2, ... :  location on the dermatome map showing surface regions of different nerves.

You should give specific factors. 'DLI' is not specific, 'DLI C1-C2' is specific. 

## Dermatome Map

1. **Cervical Dermatomes (C1-C8)**:
   - C1: At the top of the head.
   - C2: Back of the head.
   - C3: Back of the neck.
   - C4: Shoulder area.
   - C5: Lateral upper arm to the elbow.
   - C6: Forearm and the thumb side of the hand.
   - C7: Forearm and the middle finger.
   - C8: Little finger side of the hand.

2. **Thoracic Dermatomes (T1-T12)**:
   - T1: Inner side of the forearm.
   - T2-T6: Chest area.
   - T7-T12: Abdomen and lower back.

3. **Lumbar Dermatomes (L1-L5)**:
   - L1: Groin area.
   - L2: Front of the thigh.
   - L3: Knee region.
   - L4: Front and inner side of the leg.
   - L5: Outer side of the leg and the top of the foot.

4. **Sacral Dermatomes (S1-S5)**:
   - S1: Back of the leg and the heel.
   - S2-S4: Buttocks and perineal area.
   - S5: Sacrococcygeal area.

# Data

{example}

# About Output

Your output should contain parts described as follows.

**Part 0**: San Check.

How many sample are there in each groups?

**Part 1**: Consideration. 

In this part, feel free to write down the process of your considerations. 

Hint
- You need to abstract, identify, and choose suitable factors. You may write down your hypothesis.
- **Only factors in following Templates are allowed**:
   - Templates 1: **"DLI [location on the dermatome map]"**
   - Templates 2: **"[L/R] Radiculopathy [location on the dermatome map]"**

**Part 2**: Final Output.  

In this part, you are required to report the factors that you decided to use.

**Only factors in following Templates are allowed**:
- Templates 1: **"DLI [location on the dermatome map]"**
- Templates 2: **"[L/R] Radiculopathy [location on the dermatome map]"**

'''


In [19]:
propose_prompt_with_data_in_front = lambda example: f'''
You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data

{example}


# Tasks: Factor Abstraction.

**What are the high-level factors behind the text that contribute to the allocation of groups?**

Propose your candidate factors based on your observation and analysis on given data. **Only factors in following Templates are allowed**:
- Templates 1: **"DLI [location on the dermatome map]"**
- Templates 2: **"[L/R] Radiculopathy [location on the dermatome map]"**

Meaning of abbreviation:
- L: Left
- R: Right
- DLI: discoligamentous injury
- Radi: radiculopathy 
- C1, C2, ... :  location on the dermatome map showing surface regions of different nerves.

You should give specific factors. 'DLI' is not specific, 'DLI C1-C2' is specific. 

## Dermatome Map

1. **Cervical Dermatomes (C1-C8)**:
   - C1: At the top of the head.
   - C2: Back of the head.
   - C3: Back of the neck.
   - C4: Shoulder area.
   - C5: Lateral upper arm to the elbow.
   - C6: Forearm and the thumb side of the hand.
   - C7: Forearm and the middle finger.
   - C8: Little finger side of the hand.

2. **Thoracic Dermatomes (T1-T12)**:
   - T1: Inner side of the forearm.
   - T2-T6: Chest area.
   - T7-T12: Abdomen and lower back.

3. **Lumbar Dermatomes (L1-L5)**:
   - L1: Groin area.
   - L2: Front of the thigh.
   - L3: Knee region.
   - L4: Front and inner side of the leg.
   - L5: Outer side of the leg and the top of the foot.

4. **Sacral Dermatomes (S1-S5)**:
   - S1: Back of the leg and the heel.
   - S2-S4: Buttocks and perineal area.
   - S5: Sacrococcygeal area.



# About Output

Your output should contain parts described as follows.

**Part 0**: San Check.

How many sample are there in each groups?

**Part 1**: Consideration. 

In this part, feel free to write down the process of your considerations. 

Hint
- You need to abstract, identify, and choose suitable factors. You may write down your hypothesis.
- **Only factors in following Templates are allowed**:
   - Templates 1: **"DLI [location on the dermatome map]"**
   - Templates 2: **"[L/R] Radiculopathy [location on the dermatome map]"**

**Part 2**: Final Output.  

In this part, you are required to report the factors that you decided to use.

**Only factors in following Templates are allowed**:
- Templates 1: **"DLI [location on the dermatome map]"**
- Templates 2: **"[L/R] Radiculopathy [location on the dermatome map]"**

'''


In [20]:
propose_prompt_with_data_in_last = lambda example: f'''
You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Tasks: Factor Abstraction.

**What are the high-level factors behind the text that contribute to the allocation of groups?**

Propose your candidate factors based on your observation and analysis on given data. **Only factors in following Templates are allowed**:
- Templates 1: **"DLI [location on the dermatome map]"**
- Templates 2: **"[L/R] Radiculopathy [location on the dermatome map]"**

Meaning of abbreviation:
- L: Left
- R: Right
- DLI: discoligamentous injury
- Radi: radiculopathy 
- C1, C2, ... :  location on the dermatome map showing surface regions of different nerves.

You should give specific factors. 'DLI' is not specific, 'DLI C1-C2' is specific. 

## Dermatome Map

1. **Cervical Dermatomes (C1-C8)**:
   - C1: At the top of the head.
   - C2: Back of the head.
   - C3: Back of the neck.
   - C4: Shoulder area.
   - C5: Lateral upper arm to the elbow.
   - C6: Forearm and the thumb side of the hand.
   - C7: Forearm and the middle finger.
   - C8: Little finger side of the hand.

2. **Thoracic Dermatomes (T1-T12)**:
   - T1: Inner side of the forearm.
   - T2-T6: Chest area.
   - T7-T12: Abdomen and lower back.

3. **Lumbar Dermatomes (L1-L5)**:
   - L1: Groin area.
   - L2: Front of the thigh.
   - L3: Knee region.
   - L4: Front and inner side of the leg.
   - L5: Outer side of the leg and the top of the foot.

4. **Sacral Dermatomes (S1-S5)**:
   - S1: Back of the leg and the heel.
   - S2-S4: Buttocks and perineal area.
   - S5: Sacrococcygeal area.



# About Output

Your output should contain parts described as follows.

**Part 0**: San Check.

How many sample are there in each groups?

**Part 1**: Consideration. 

In this part, feel free to write down the process of your considerations. 

Hint
- You need to abstract, identify, and choose suitable factors. You may write down your hypothesis.
- **Only factors in following Templates are allowed**:
   - Templates 1: **"DLI [location on the dermatome map]"**
   - Templates 2: **"[L/R] Radiculopathy [location on the dermatome map]"**

**Part 2**: Final Output.  

In this part, you are required to report the factors that you decided to use.

**Only factors in following Templates are allowed**:
- Templates 1: **"DLI [location on the dermatome map]"**
- Templates 2: **"[L/R] Radiculopathy [location on the dermatome map]"**


# Data

{example}


'''


# Ground truth

In [21]:
all_node_list = [f for f in data_interface.columns if ('DLI' in f) or ('Radi' in f)]

In [22]:
gt_causal_parents = ['R C4 Radiculopathy', 'R C5 Radiculopathy', 'R C6 Radiculopathy']
gt_causal_ancestors = ['DLI C3-C4', 'DLI C4-C5', 'DLI C5-C6'] + gt_causal_parents

In [23]:
def get_possible_ancestors(G, annotated_name):
    d = G.shape[0]

    # A[i,j] = 1 iff there is possibly a edge from j to i
    A = np.zeros((d,d))
    for i in range(d):
        for j in range(d):
            if i==j:
                continue
            if (G[i,j]==-1) and (G[j,i]==1): # i --> j
                A[j,i] = 1

            if (G[i,j]==2) and (G[j,i]==1): # i o-> j
                A[j,i] = 1

            if (G[i,j]==2) and (G[j,i]==2): # i o-o j
                A[j,i] = 1
                A[i,j] = 1

            if (G[i,j]==1) and (G[j,i]==1): # i <-> j
                pass 

    possible_path = A>0
    for _ in range(d):
        possible_path |= (possible_path@A)>0

    return [annotated_name[idx] for idx in range(d) if possible_path[0,idx]]

In [24]:
def Eval_Factor_Discovery(factor_set):
    num_parents = 0
    num_ancestors = 0
    num_others = 0
    for fct in factor_set:
        if fct in gt_causal_ancestors:
            num_ancestors += 1
        else:
            num_others += 1
        if fct in gt_causal_parents:
            num_parents += 1  
    return [num_parents, num_ancestors, num_others]

In [25]:
def get_possible_parents(g, annotated_name):
    possible_parents = np.array([False] * len(g.graph[:,0]))

    # o-> Y
    possible_parents |= (g.graph[:,0]==2) & (g.graph[0,:] == 1)
    # --> Y
    possible_parents |= (g.graph[:,0]==-1) & (g.graph[0,:] == 1)
    # o-o Y
    possible_parents |= (g.graph[:,0]==2) & (g.graph[0,:] == 2)

    return [str(s) for s in np.array(annotated_name)[possible_parents]]

In [26]:
def Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'):

    node_set = set(annotated_name)
    if mode == 'ancestors':
        pos_factor_set = gt_causal_ancestors
    elif mode == 'parents':
        pos_factor_set = gt_causal_parents
        
    ground_truth = np.array([ 1 if f in pos_factor_set else 0 for f in all_node_list])
    estimated = np.array([ 1 if f in possible_ancestors else 0 for f in all_node_list])

    recall = (ground_truth*estimated).sum() / ground_truth.sum()
    accuracy = (ground_truth==estimated).mean()

    precision, F1 = '-', '-'
    if estimated.sum() > 0:
        precision = (ground_truth*estimated).sum() / estimated.sum()
        if recall*precision > 0:
            F1 = 2/(1/recall + 1/precision)

    return [recall, accuracy, F1, precision] 

# Llama-2-70b

https://poe.com/Llama-2-70b


log https://poe.com/s/ewbCHyuyGEKcbY4TqWuZ

In [19]:
V = set(['R shoulder impingement'])
all_factors = set(['R shoulder impingement'])
factors = [ f for f in V if f!= target_name]
factors

[]

## iteration 1

In [72]:
print(propose_prompt_with_data_in_front(init_data))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data

 

### Group with 'R shoulder impingement'=0

----


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impingement, accompanied by non-specific problems with the right shoulder. The patient also reports issues in the lower body, including concerns with the left adductor tendon, unspecified lumbago, discomfort in the left hip joint, and pain in the right shin. Further symptoms involve trouble with the left knee, pain in the left hamstring, issues with the left toe, and pain located in the left rear thigh.

----


Patient ID: Neuro_00040
Age: 19
Gender: Male

Symptom Diagnosis:
The patient presents with a range of symptoms predominantly relating to the lower back and left lowe

### feedback

In [20]:

input_factors = '''  
DLI C1-C2
L Radiculopathy C5
R Radiculopathy C6
DLI T1-T2
L Radiculopathy L2
R Radiculopathy L5

DLI C1-C2 (discoligamentous injury in the cervical region)
L Radiculopathy C4-C5 (left radiculopathy in the cervical region)
R Radiculopathy C4-C5 (right radiculopathy in the cervical region)

DLI C1-C2
L Radiculopathy C4-C5
R Radiculopathy C4-C5
L Radiculopathy L5
R Radiculopathy L5

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['DLI T1-T2',
 'DLI T12-L1',
 'DLI C1-C2',
 'R L5 Radiculopathy',
 'R C4 Radiculopathy',
 'R C6 Radiculopathy',
 'L C4 Radiculopathy',
 'DLI T10-T11',
 'DLI T11-T12',
 'R C5 Radiculopathy',
 'L C5 Radiculopathy',
 'L L5 Radiculopathy']

### confirmation

In [21]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)

R C4 Radiculopathy
R C6 Radiculopathy
R C5 Radiculopathy
L C5 Radiculopathy


In [22]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 4: 100%|██████████| 5/5 [00:00<00:00, 950.23it/s]

[0.36319974 0.4166811  0.4356054  0.34220843 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="R C4 Radiculopathy"];
2 [label=X3];
2 [label="R C6 Radiculopathy"];
3 [label=X4];
3 [label="L C5 Radiculopathy"];
4 [label=X5];
4 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [28]:
# meta_data
print(*Eval_Factor_Discovery(factors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, factors, mode = 'ancestors'), sep='\t')

3	3	1
0.5	0.9487179487179487	0.6000000000000001	0.75


In [25]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['R C4 Radiculopathy', 'R C6 Radiculopathy', 'L C5 Radiculopathy', 'R C5 Radiculopathy']
3	3	1
0.5	0.9487179487179487	0.6000000000000001	0.75


In [26]:
possible_parents = get_possible_parents(g, annotated_name)
print(possible_parents)
print(*Eval_Factor_Discovery(possible_parents), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_parents, mode = 'parents'), sep='\t')

['R C4 Radiculopathy', 'R C6 Radiculopathy', 'L C5 Radiculopathy', 'R C5 Radiculopathy']
3	3	1
1.0	0.9871794871794872	0.8571428571428572	0.75


## iteration 2

### feedback

In [29]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.5908422462755826 18 [0.72222222 0.27777778]
1 0.6881388137135884 20 [0.55 0.45]
2 0.11466528220290656 41 [0.97560976 0.02439024]
3 0.5488743844849944 21 [0.76190476 0.23809524]
choose g==[0, 1, 2, 3]


In [119]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00071
Age: 44
Gender: Male

Symptom Diagnosis:
The patient presents with a constellation of symptoms that include problems with the left neck, right shoulder, and right ham coupled with right shoulder trouble with specific difficulty in performing a 'thumbs up.' The patient also reports trouble in the left hip joint, lower left leg, and left ankle, in addition to right obesity and upper abdominal discomfort.

---


Patient ID: Neuro_00019
Age: 57
Gender: Female

Symptom Diagnosis:
The patient presents with a constellation of symptoms involving bilateral shoulders with impingement syndrome primarily on the left shoulder, left neck problems, generalized left arm issues, and left shoulder trouble. Furthermore, the patient experiences discomfort in the lower back region acknowledged as left lumbago, r

In [120]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00035
Age: 55
Gender: Female

Symptom Diagnosis Summary:
The patient presents with a complex set of symptoms including neck pain, discomfort in the right ear, headaches, and a sensation of pharyngeal discomfort. Additionally, the patient is experiencing central chest disorders on the left side, bilateral shoulder problems, as well as trouble more specifically in the left shoulder. She also reports discomfort in the left arm and in the right bend of the arm. These symptoms suggest involvement of multiple anatomical structures and potential neural pathways.

---


Patient ID: Neuro_00094
Age: 49
Gender: Female

Symptom diagnosis summary:
The patient presents with diverse musculoskeletal symptoms predominantly affecting the neck, shoulders, and back, accompanied by localized troubles. Specifically, s

In [121]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00041
Age: 43
Gender: Male

Symptom Diagnosis:
The patient presents with lumbago characterized by lower back pain, as well as radiating pain to the left lower extremity (L pta), pain in the left hip joint, and trouble with both the left and right ankles. Additionally, the patient has a concern regarding right-sided obesity which may be contributing to the musculoskeletal symptoms.

---


Patient ID: Neuro_00063
Age: 14
Gender: Male

Symptom diagnosis summary:
The patient presents with a right hip arthritis, indicating inflammation and possible degenerative changes within the right hip joint. This is characterized by pain, limited range of motion, and potential swelling. Additionally, the patient is exhibiting symptoms consistent with a left medial knee joint disorder, which may involve the medial 

In [122]:
example = ''

chosen_idx = g==g_chosen[3]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00099
Age: 28
Gender: Female

Symptom Diagnosis Summary:
The patient presented with a range of symptoms affecting various bilateral anatomical regions. She reports having trouble with the right (R) jaw and right shoulder, which could be indicative of joint or musculoskeletal issues in these areas. Additionally, she is experiencing left (L) lateral elbow pain and left wrist problems, both of which could be related to overuse injuries or strain. Furthermore, she describes discomfort related to the left adductor tendon, suggesting a possible tendinopathy or strain. The patient also reports lumbago, a general term for lower back pain, which may result from a variety of underlying conditions. Obesity on the right side presents an atypical clinical picture that requires further investigation. She is als

In [30]:
input_factors = '''  
0.
DLI C1-C4 (discoligamentous injury in the cervical spine)
Radiculopathy C5-C8 (radiculopathy in the cervical spine)
DLI T1-T4 (discoligamentous injury in the thoracic spine)
Radiculopathy T7-T12 (radiculopathy in the thoracic spine)
DLI L1-L5 (discoligamentous injury in the lumbar spine)
Radiculopathy L2-L5 (radiculopathy in the lumbar spine)

DLI C1-C2
Radiculopathy C4-C5
DLI C5-C6
Radiculopathy C6-C7

DLI C1-C2
L Radiculopathy C5
R Radiculopathy C6
R Shoulder Impingement


1.
DLI C1-C2 (Discoligamentous Injury in the C1-C2 region)
Radiculopathy C4 (Radiculopathy in the C4 region)
Cervical Dermatomes C4 (Symptoms in the C4 dermatome)
Thoracic Dermatomes T2-T6 (Symptoms in the T2-T6 dermatome)
Lumbar Dermatomes L2-L4 (Symptoms in the L2-L4 dermatome)

DLI C1-C2
L Radiculopathy C4-C5
R Radiculopathy C5-C6

DLI C4-C5 (Cervical Dermatomes C4-C5)
Radiculopathy C4-C5 (Cervical Radiculopathy C4-C5)
DLI T1-T2 (Thoracic Dermatomes T1-T2)
Radiculopathy T1-T2 (Thoracic Radiculopathy T1-T2)

2.
DLI C1-C2
L Radiculopathy C5
R Radiculopathy C4
L Radiculopathy L5
R Radiculopathy L4

DLI C1-C2
L Radiculopathy C4-C5
R Radiculopathy C5-C6

DLI C1-C2
L Radiculopathy C4-C5
R Radiculopathy C4-C5
DLI L2-L3
L Radiculopathy L4-L5
R Radiculopathy L4-L5

3.
DLI C4-C5 (Right shoulder impingement)
DLI C6-C7 (Right lateral arm and forearm)
Radiculopathy C4-C5 (Right shoulder and arm)

DLI C1-C2 (discoligamentous injury in the cervical region)
L Radiculopathy C5-C6 (left radiculopathy in the cervical region)
R Radiculopathy C5-C6 (right radiculopathy in the cervical region)
L Radiculopathy T1-T2 (left radiculopathy in the thoracic region)
R Radiculopathy T1-T2 (right radiculopathy in the thoracic region)

DLI C4-C5
L Radiculopathy C5-C6
R Radiculopathy C5-C6
DLI C6-C7
L Radiculopathy C7-C8
R Radiculopathy C7-C8

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['DLI L4-L5',
 'DLI T3-T4',
 'L T10 Radiculopathy',
 'L L4 Radiculopathy',
 'L C6 Radiculopathy',
 'R T7 Radiculopathy',
 'R C8 Radiculopathy',
 'DLI L2-L3',
 'R L4 Radiculopathy',
 'DLI C6-C7',
 'L T7 Radiculopathy',
 'R T10 Radiculopathy',
 'DLI C5-C6',
 'DLI C3-C4',
 'L C8 Radiculopathy',
 'L C7 Radiculopathy',
 'R C7 Radiculopathy',
 'DLI L1-L2',
 'DLI C4-C5']

### confirmation

In [31]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

L C6 Radiculopathy
R C6 Radiculopathy
L C5 Radiculopathy
R C5 Radiculopathy
DLI C5-C6
DLI C3-C4
R C4 Radiculopathy
L C7 Radiculopathy
R C7 Radiculopathy
DLI C4-C5
False


In [32]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 10: 100%|██████████| 11/11 [00:00<00:00, 643.83it/s]

[0.36319974 0.46921774 0.4166811  0.40270162 0.4356054  0.38644277
 0.4        0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="L C7 Radiculopathy"];
4 [label=X5];
4 [label="R C6 Radiculopathy"];
5 [label=X6];
5 [label="R C7 Radiculopathy"];
6 [label=X7];
6 [label="DLI C3-C4"];
7 [label=X8];
7 [label="L C6 Radiculopathy"];
8 [label=X9];
8 [label="L C5 Radiculopathy"];
9 [label=X10];
9 [label="DLI C4-C5"];
10 [label=X11];
10 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
9 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 5  [arrowhead=odot, arrowtail=odo

In [33]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'DLI C3-C4', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	6	2
1.0	0.9743589743589743	0.8571428571428572	0.75


## iteration 3

### feedback

In [35]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(background_factor), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.410116318288409 14 [0.85714286 0.14285714]
1 0.666278442414676 13 [0.61538462 0.38461538]
2 0.6108643020548935 30 [0.7 0.3]
3 0.3094772867741259 43 [0.90697674 0.09302326]
choose g==[0, 1, 2, 3]


In [178]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impingement, accompanied by non-specific problems with the right shoulder. The patient also reports issues in the lower body, including concerns with the left adductor tendon, unspecified lumbago, discomfort in the left hip joint, and pain in the right shin. Further symptoms involve trouble with the left knee, pain in the left hamstring, issues with the left toe, and pain located in the left rear thigh.

---


Patient ID: Neuro_00061
Age: 29
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a complex array of symptoms which include pain localized to the righ

In [179]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---



---


Patient id: Neuro_00068
Age: 42
Gender: Male

Symptom diagnosis:
The patient presents with a range of musculoskeletal symptoms including left neck problems, complications with both shoulders, and additional trouble in the left and right hands. The patient describes specific difficulties with the left small finger, alongside issues with both medial elbows. These varied symptoms are suggestive of a multifocal neurological or musculoskeletal condition affecting multiple limbs and warrant thorough investigation.

---


Patient ID: Neuro_00044
Age: 33
Gender: Female

Summary of Symptom Diagnosis:
The patient has presented with pain localized to the neck and has been experiencing issues with both shoulders. Specifically, there is documentation of problems in the left shoulder as well as trouble in the right should

In [180]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00073 
Age: 12 
Gender: Female

Symptom Diagnosis:
The patient presents with a range of musculoskeletal complaints including trouble in the right ear, headaches, issues in both shoulders (left and right), with particular trouble within the shoulder capsule. She is experiencing lateral pain on her right elbow, unspecified problems with the left arm, medial elbow problems on the right, bilateral anterior knee pain, and difficulty with the left ankle. Additionally, the patient has left-sided obesity and issues with the right tear duct.

---


Patient ID: Neuro_00037
Age: 11
Gender: Female

Symptom Diagnosis Summary:
The patient is presenting with shoulder problems affecting both the left and right shoulders. The specific nature and characteristics of the shoulder issues have not been detailed in the 

In [181]:
example = ''

chosen_idx = g==g_chosen[3]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00051
Age: 24
Gender: Male

Summary of Symptom Diagnosis:
The patient presents with a range of symptoms, including right-sided tinnitus, general headache, pharyngeal discomfort, right hip arthritis, and pain in the right front knee. These symptoms collectively suggest a constellation of neurologic and musculoskeletal issues that require further investigation to determine their etiology and interrelation.

---


Patient ID: Neuro_00063
Age: 14
Gender: Male

Symptom diagnosis summary:
The patient presents with a right hip arthritis, indicating inflammation and possible degenerative changes within the right hip joint. This is characterized by pain, limited range of motion, and potential swelling. Additionally, the patient is exhibiting symptoms consistent with a left medial knee joint disorder, which

In [36]:
input_factors = '''  
0.
DLI C5-C6 (discoligamentous injury in the upper limb region)
Radiculopathy C5-C6 (radiculopathy in the upper limb region)

DLI C3-C4 (discoligamentous injury between C3 and C4 vertebrae)
DLI C5-C6 (discoligamentous injury between C5 and C6 vertebrae)
L Radiculopathy C4 (radiculopathy affecting the left C4 nerve root)
R Radiculopathy C5 (radiculopathy affecting the right C5 nerve root)

DLI C4-C5 (Right shoulder impingement)
Radiculopathy C5 (Right upper arm and forearm)
DLI C6-C7 (Right forearm and hand)
Radiculopathy C7 (Right hand)

1.
DLI C1-C2
Radiculopathy C4-C5

DLI C4-C5
Radiculopathy C5-C6

DLI C1-C2
R Radiculopathy C5
L Radiculopathy C6

2.
DLI C4-C5 (Right shoulder impingement)
DLI C5-C6 (Left shoulder impingement)
L Radiculopathy C5-C6 (Left shoulder radiculopathy)
R Radiculopathy C5-C6 (Right shoulder radiculopathy)

DLI C4-C5 (Right shoulder impingement)
DLI C5-C6 (Left shoulder impingement)
L Radiculopathy C5 (Left radial nerve)
R Radiculopathy C5 (Right radial nerve)

DLI C1-C2
DLI C3-C4
L Radiculopathy C5-C6
R Radiculopathy C5-C6

3.
DLI C1-C2
L Radiculopathy C5
R Radiculopathy C6
L Shoulder Impingement
R Shoulder Impingement

DLI C1-C2
L Radiculopathy C5
R Radiculopathy C6
L Shoulder Impingement
R Shoulder Impingement

DLI C1-C2
L Radiculopathy C5
R Radiculopathy C6
L Lower Back Pain
R Lower Back Pain
R Hip Arthritis
L Ankle Trouble

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

[]

### confirmation

In [37]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

L C6 Radiculopathy
R C6 Radiculopathy
L C5 Radiculopathy
R C5 Radiculopathy
DLI C5-C6
DLI C3-C4
R C4 Radiculopathy
L C7 Radiculopathy
R C7 Radiculopathy
DLI C4-C5
True


In [38]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False, independence_test_method='fisherz')

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 10: 100%|██████████| 11/11 [00:00<00:00, 640.50it/s]

[0.36319974 0.46921774 0.4166811  0.40270162 0.4356054  0.38644277
 0.4        0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="L C7 Radiculopathy"];
4 [label=X5];
4 [label="R C6 Radiculopathy"];
5 [label=X6];
5 [label="R C7 Radiculopathy"];
6 [label=X7];
6 [label="DLI C3-C4"];
7 [label=X8];
7 [label="L C6 Radiculopathy"];
8 [label=X9];
8 [label="L C5 Radiculopathy"];
9 [label=X10];
9 [label="DLI C4-C5"];
10 [label=X11];
10 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
9 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 5  [arrowhead=odot, arrowtail=odo

In [39]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'DLI C3-C4', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	6	2
1.0	0.9743589743589743	0.8571428571428572	0.75


## Llama-2-70b Zeroshot

https://poe.com/s/wxqtxn6bHn0v9nvbVzwy

https://poe.com/s/T5D4GLwsZg7XsjqcFonG

In [40]:
input_factors = '''  
DLI C4-C5: Discoligamentous injury between C4 and C5 vertebrae, which may cause compression or irritation of the nerves emerging from the cervical spine and innervating the shoulder region.
R Radiculopathy C5: Right radiculopathy at the C5 level, which may cause compression or irritation of the nerve root emerging from the cervical spine and innervating the shoulder region.
DLI C5-C6: Discoligamentous injury between C5 and C6 vertebrae, which may cause compression or irritation of the nerves emerging from the cervical spine and innervating the shoulder region.
R Radiculopathy C6: Right radiculopathy at the C6 level, which may cause compression or irritation of the nerve root emerging from the cervical spine and innervating the shoulder region.

DLI C4-C5
R Radiculopathy C5
DLI C5-C6
R Radiculopathy C6

DLI C4-C5
L Radiculopathy C4-C5
L Radiculopathy T1-T2


'''.split('\n')
proposed_zero_shot_factors = get_variables_with_data(input_factors)
proposed_zero_shot_factors

['L T10 Radiculopathy',
 'DLI C5-C6',
 'L C4 Radiculopathy',
 'L T11 Radiculopathy',
 'R C6 Radiculopathy',
 'DLI C6-C7',
 'L T1 Radiculopathy',
 'L T12 Radiculopathy',
 'L C5 Radiculopathy',
 'DLI C4-C5',
 'L T2 Radiculopathy',
 'R C5 Radiculopathy']

In [41]:
proposed_zero_shot_factors = [
 'DLI C6-C7',
 'L T2 Radiculopathy',
 'DLI C5-C6',
 'R C5 Radiculopathy',
 'L T1 Radiculopathy',
 'L C4 Radiculopathy',
 'DLI C4-C5',
 'R C6 Radiculopathy',
 'L C5 Radiculopathy']

In [42]:
print(*Eval_Factor_Discovery(proposed_zero_shot_factors), sep='\t')
print(*Eval_Factor_Confirmation(proposed_zero_shot_factors, proposed_zero_shot_factors, mode = 'ancestors'), sep='\t')

2	4	5
0.6666666666666666	0.9102564102564102	0.5333333333333333	0.4444444444444444


# GPT-3.5-Turbo

https://poe.com/GPT-3.5-Turbo

https://poe.com/s/OdNqcJfHCUyHWeFhE7C7

In [43]:
V = set(['R shoulder impingement'])
all_factors = set(['R shoulder impingement'])
factors = [ f for f in V if f!= target_name]
factors

[]

## iteration 1

In [189]:
print(propose_prompt_with_data_in_front(init_data))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data

 

### Group with 'R shoulder impingement'=0

----


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impingement, accompanied by non-specific problems with the right shoulder. The patient also reports issues in the lower body, including concerns with the left adductor tendon, unspecified lumbago, discomfort in the left hip joint, and pain in the right shin. Further symptoms involve trouble with the left knee, pain in the left hamstring, issues with the left toe, and pain located in the left rear thigh.

----


Patient ID: Neuro_00040
Age: 19
Gender: Male

Symptom Diagnosis:
The patient presents with a range of symptoms predominantly relating to the lower back and left lowe

### feedback

In [44]:

input_factors = '''  
DLI C4
DLI C5
Radiculopathy C6
Radiculopathy L4
Radiculopathy S1
Radiculopathy S5
Radiculopathy C2 (only for the group with 'R shoulder impingement'=1)
Radiculopathy T7-T12 (only for the group with 'R shoulder impingement'=1)

Radiculopathy C5-C7
Radiculopathy L4-L5
Radiculopathy S1
Radiculopathy C4
Radiculopathy T7-T12

Radiculopathy C5-C7
Radiculopathy L4-L5
Radiculopathy S1
Radiculopathy C4
Radiculopathy T7-T12
'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['L S1 Radiculopathy',
 'L T10 Radiculopathy',
 'L L4 Radiculopathy',
 'L C6 Radiculopathy',
 'R T7 Radiculopathy',
 'R L4 Radiculopathy',
 'R C6 Radiculopathy',
 'L C5 Radiculopathy',
 'L T7 Radiculopathy',
 'R C5 Radiculopathy',
 'R T10 Radiculopathy',
 'DLI C5-C6',
 'L C4 Radiculopathy',
 'R C2 Radiculopathy',
 'R S1 Radiculopathy',
 'R L5 Radiculopathy',
 'R C4 Radiculopathy',
 'L C7 Radiculopathy',
 'R C7 Radiculopathy',
 'DLI C4-C5',
 'L L5 Radiculopathy']

### confirmation

In [45]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)

L C6 Radiculopathy
R C6 Radiculopathy
L C5 Radiculopathy
R C5 Radiculopathy
DLI C5-C6
R C4 Radiculopathy
L C7 Radiculopathy
R C7 Radiculopathy
DLI C4-C5


In [46]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 9: 100%|██████████| 10/10 [00:00<00:00, 691.79it/s]

[0.36319974 0.46921774 0.4166811  0.40270162 0.4356054  0.38644277
 0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="L C7 Radiculopathy"];
4 [label=X5];
4 [label="R C6 Radiculopathy"];
5 [label=X6];
5 [label="R C7 Radiculopathy"];
6 [label=X7];
6 [label="L C6 Radiculopathy"];
7 [label=X8];
7 [label="L C5 Radiculopathy"];
8 [label=X9];
8 [label="DLI C4-C5"];
9 [label=X10];
9 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
8 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
4 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
7 -> 8  [arrowhead=odot, arrowtail=od

In [47]:
# meta_data
print(*Eval_Factor_Discovery(factors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, factors, mode = 'ancestors'), sep='\t')

3	5	4
0.8333333333333334	0.9358974358974359	0.6666666666666666	0.5555555555555556


In [193]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['L C5 Radiculopathy', 'DLI C5-C6', 'R C5 Radiculopathy', 'L C6 Radiculopathy', 'DLI C4-C5', 'R C6 Radiculopathy', 'R C4 Radiculopathy']
3	5	2
0.8333333333333334	0.9615384615384616	0.7692307692307694	0.7142857142857143


In [194]:
possible_parents = get_possible_parents(g, annotated_name)
print(possible_parents)
print(*Eval_Factor_Discovery(possible_parents), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_parents, mode = 'parents'), sep='\t')

['DLI C5-C6', 'DLI C4-C5', 'R C4 Radiculopathy']
1	3	0
0.3333333333333333	0.9487179487179487	0.3333333333333333	0.3333333333333333


## iteration 2

### feedback

In [48]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.5982695885852573 21 [0.71428571 0.28571429]
1 0.5890026049647302 29 [0.72413793 0.27586207]
2 0.6931471805599453 12 [0.5 0.5]
3 0.0 38 [1.]
choose g==[0, 1, 2]


In [196]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00028
Age: 55
Gender: Male

SYMPTOM DIAGNOSIS:
The patient presents with a multifocal array of symptoms including headaches localized to the forehead, right side, and the back of the head. Regional pain extends to both shoulders with noted trouble, specifically the left medial elbow, left hand, and finger trouble. The patient reports lumbago, pain in the left hip joint, left shin discomfort, and issues with the right ankle. Right-sided obesity is documented alongside a disorder in the left dorsal knee joint and right Achilles tendon problems.

---


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impinge

In [197]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00046
Age: 27
Gender: Female

Symptom Diagnosis Summary:
The patient presents with musculoskeletal complaints encompassing the cervical, shoulder, lumbar, and lower extremity regions. Specifically, there is neck pain localized to the right neck, indicating a possible unilateral cervical issue. The left shoulder exhibits impingement and other unspecified problems, while the right wrist and hand are also symptomatic. Complaints include pain in the left adductor tendon and bilateral lumbago, with a particular emphasis on the right side. The patient additionally reports obesity, which might be contributing to the lower limb issues, like the left dorsal knee joint disorder, right heel discomfort, and complications with the left Achilles tendon. 

---


Patient ID: Neuro_00088
Age: 52
Gender: Male

Summ

In [198]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient id: Neuro_00068
Age: 42
Gender: Male

Symptom diagnosis:
The patient presents with a range of musculoskeletal symptoms including left neck problems, complications with both shoulders, and additional trouble in the left and right hands. The patient describes specific difficulties with the left small finger, alongside issues with both medial elbows. These varied symptoms are suggestive of a multifocal neurological or musculoskeletal condition affecting multiple limbs and warrant thorough investigation.

---


Patient ID: Neuro_00014
Age: 20
Gender: Male

Symptom Diagnosis Summary:
The patient presents with pain predominantly in the cervical and lumbosacral regions, manifested as neck pain, with specific complaints in the right neck area. Additionally, the patient reports issues with the right front axle, whic

In [49]:
input_factors = '''  
0.
R shoulder impingement
Specific locations of pain mentioned in the symptom diagnoses.

DLI C4-C5
L Radiculopathy C5
R Radiculopathy C5
L Radiculopathy C6
R Radiculopathy C6

DLI C4
L Radiculopathy C5
L Radiculopathy C6
L Radiculopathy C7
L Radiculopathy L4
R Radiculopathy C5
R Radiculopathy C6
R Radiculopathy C7


1.
DLI C4
L/R Radiculopathy C5
L/R Radiculopathy C6
L/R Radiculopathy C7
L/R Radiculopathy C8

DLI C4 (group with 'R shoulder impingement' = 0): This factor indicates the presence of a discoligamentous injury in the C4 dermatome region, which is associated with symptoms in the shoulder area.
R Radiculopathy C5 (group with 'R shoulder impingement' = 1): This factor indicates the presence of right-sided radiculopathy in the C5 dermatome region, which is associated with symptoms in the lateral upper arm to the elbow.

DLI C4
Radiculopathy C5
Radiculopathy C6
Radiculopathy L5
Radiculopathy S1

2.
DLI C4
L Radiculopathy C6
R Radiculopathy C6
L Radiculopathy C7
R Radiculopathy C7
L Radiculopathy C8
R Radiculopathy C8

DLI C4 (group with 'R shoulder impingement' = 0): The presence of discoligamentous injury in the C4 dermatome region.
L Radiculopathy C5-C8 (group with 'R shoulder impingement' = 0): The presence of left radiculopathy in the C5-C8 dermatome regions.
DLI C4 (group with 'R shoulder impingement' = 1): The presence of discoligamentous injury in the C4 dermatome region.
R Radiculopathy C5-C8 (group with 'R shoulder impingement' = 1): The presence of right radiculopathy in the C5-C8 dermatome regions.

DLI C4 (group with 'R shoulder impingement' = 0): The presence of discoligamentous injury in the C4 dermatome region.
L Radiculopathy C5-C8 (group with 'R shoulder impingement' = 0): The presence of left radiculopathy in the C5-C8 dermatome regions.
DLI C4 (group with 'R shoulder impingement' = 1): The presence of discoligamentous injury in the C4 dermatome region.
R Radiculopathy C5-C8 (group with 'R shoulder impingement' = 1): The presence of right radiculopathy in the C5-C8 dermatome regions.

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['R C8 Radiculopathy', 'L C8 Radiculopathy']

### confirmation

In [50]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

L C6 Radiculopathy
R C6 Radiculopathy
L C5 Radiculopathy
R C5 Radiculopathy
DLI C5-C6
R C4 Radiculopathy
L C7 Radiculopathy
R C7 Radiculopathy
DLI C4-C5
True


In [51]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 9: 100%|██████████| 10/10 [00:00<00:00, 680.99it/s]

[0.36319974 0.46921774 0.4166811  0.40270162 0.4356054  0.38644277
 0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="L C7 Radiculopathy"];
4 [label=X5];
4 [label="R C6 Radiculopathy"];
5 [label=X6];
5 [label="R C7 Radiculopathy"];
6 [label=X7];
6 [label="L C6 Radiculopathy"];
7 [label=X8];
7 [label="L C5 Radiculopathy"];
8 [label=X9];
8 [label="DLI C4-C5"];
9 [label=X10];
9 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
8 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
4 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
7 -> 8  [arrowhead=odot, arrowtail=od

In [52]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	5	2
0.8333333333333334	0.9615384615384616	0.7692307692307694	0.7142857142857143


## GPT-3.5 Zeroshot

https://poe.com/s/wxqtxn6bHn0v9nvbVzwy

In [53]:
input_factors = '''  

DLI C4-C8
Radiculopathy C4-C8

DLI C4-C7: Discoligamentous injury in the cervical spine, specifically affecting the C4, C5, C6, and C7 dermatomes, can lead to right shoulder impingement. This injury may result in nerve compression or irritation, causing pain and dysfunction in the shoulder region.
R Radiculopathy C5-C7: Right-sided radiculopathy originating from the C5, C6, and C7 nerve roots can contribute to right shoulder impingement. Radiculopathy refers to the compression or irritation of a nerve root, leading to symptoms along the nerve pathway. In this case, the affected nerve roots can cause pain, weakness, and sensory changes in the right shoulder, potentially contributing to impingement.

DLI C4-C7
R Radiculopathy C4-C7

'''.split('\n')
proposed_zero_shot_factors = get_variables_with_data(input_factors)
proposed_zero_shot_factors

['R C8 Radiculopathy',
 'DLI C5-C6',
 'R C4 Radiculopathy',
 'L C4 Radiculopathy',
 'R C6 Radiculopathy',
 'DLI C7-C8',
 'R C7 Radiculopathy',
 'DLI C6-C7',
 'DLI C4-C5',
 'L C8 Radiculopathy',
 'R C5 Radiculopathy']

In [54]:
print(*Eval_Factor_Discovery(proposed_zero_shot_factors), sep='\t')
print(*Eval_Factor_Confirmation(proposed_zero_shot_factors, proposed_zero_shot_factors, mode = 'ancestors'), sep='\t')

3	5	6
0.8333333333333334	0.9102564102564102	0.588235294117647	0.45454545454545453


# Llama-2-13b

https://poe.com/Llama-2-13b

https://poe.com/s/WhItBip5QNtm1eXlSKNP

In [74]:
V = set(['R shoulder impingement'])
all_factors = set(['R shoulder impingement'])
factors = [ f for f in V if f!= target_name]
factors

[]

## iteration 1

In [75]:
print(propose_prompt_with_data_in_front(init_data))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data

 

### Group with 'R shoulder impingement'=0

----


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impingement, accompanied by non-specific problems with the right shoulder. The patient also reports issues in the lower body, including concerns with the left adductor tendon, unspecified lumbago, discomfort in the left hip joint, and pain in the right shin. Further symptoms involve trouble with the left knee, pain in the left hamstring, issues with the left toe, and pain located in the left rear thigh.

----


Patient ID: Neuro_00040
Age: 19
Gender: Male

Symptom Diagnosis:
The patient presents with a range of symptoms predominantly relating to the lower back and left lowe

### feedback

In [76]:

input_factors = '''  
DLI C1-C2: Right shoulder impingement and left shoulder impingement
Radiculopathy L1-L5: Lower back pain and leg pain

DLI C4-C5 (Left shoulder impingement)
Radiculopathy L2-L3 (Left lumbar radiculopathy)
DLI C6-C7 (Left wrist and hand)

DLI (location on the dermatome map): C1-C2, C3-C4, C5-C6, C7-C8, L1-L5, and S1-S5.
Radiculopathy (location on the dermatome map): L/R C1-C2, L/R C3-C4, L/R C5-C6, L/R C7-C8, L/R L1-L5, and L/R S1-S5.

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['DLI L4-L5',
 'L S1 Radiculopathy',
 'R L1 Radiculopathy',
 'L L1 Radiculopathy',
 'L C6 Radiculopathy',
 'R C8 Radiculopathy',
 'R C6 Radiculopathy',
 'DLI C6-C7',
 'L L3 Radiculopathy',
 'L C5 Radiculopathy',
 'R C5 Radiculopathy',
 'DLI C5-C6',
 'L C4 Radiculopathy',
 'R C2 Radiculopathy',
 'R S1 Radiculopathy',
 'DLI C3-C4',
 'DLI S1-S2',
 'L C8 Radiculopathy',
 'R C3 Radiculopathy',
 'DLI C1-C2',
 'R L5 Radiculopathy',
 'R C4 Radiculopathy',
 'L C7 Radiculopathy',
 'L C3 Radiculopathy',
 'DLI C7-C8',
 'R C7 Radiculopathy',
 'L C2 Radiculopathy',
 'DLI L1-L2',
 'L L5 Radiculopathy',
 'DLI C4-C5']

### confirmation

In [77]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

L C6 Radiculopathy
R C6 Radiculopathy
L C5 Radiculopathy
R C5 Radiculopathy
DLI C5-C6
DLI C3-C4
R C4 Radiculopathy
L C7 Radiculopathy
R C7 Radiculopathy
DLI C4-C5
False


In [78]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 10: 100%|██████████| 11/11 [00:00<00:00, 675.81it/s]

[0.36319974 0.46921774 0.4166811  0.40270162 0.4356054  0.38644277
 0.4        0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="L C7 Radiculopathy"];
4 [label=X5];
4 [label="R C6 Radiculopathy"];
5 [label=X6];
5 [label="R C7 Radiculopathy"];
6 [label=X7];
6 [label="DLI C3-C4"];
7 [label=X8];
7 [label="L C6 Radiculopathy"];
8 [label=X9];
8 [label="L C5 Radiculopathy"];
9 [label=X10];
9 [label="DLI C4-C5"];
10 [label=X11];
10 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
9 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 5  [arrowhead=odot, arrowtail=odo

In [80]:
# meta_data
print(*Eval_Factor_Discovery(factors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, factors, mode = 'ancestors'), sep='\t')

3	6	4
1.0	0.9487179487179487	0.7499999999999999	0.6


In [79]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'DLI C3-C4', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	6	2
1.0	0.9743589743589743	0.8571428571428572	0.75


## iteration 2

### feedback

In [81]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.410116318288409 14 [0.85714286 0.14285714]
1 0.666278442414676 13 [0.61538462 0.38461538]
2 0.6108643020548935 30 [0.7 0.3]
3 0.3094772867741259 43 [0.90697674 0.09302326]
choose g==[0, 1, 2, 3]


In [103]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impingement, accompanied by non-specific problems with the right shoulder. The patient also reports issues in the lower body, including concerns with the left adductor tendon, unspecified lumbago, discomfort in the left hip joint, and pain in the right shin. Further symptoms involve trouble with the left knee, pain in the left hamstring, issues with the left toe, and pain located in the left rear thigh.

---


Patient ID: Neuro_00093
Age: 47
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a range of musculoskeletal complaints, including left neck problems 

In [104]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00094
Age: 49
Gender: Female

Symptom diagnosis summary:
The patient presents with diverse musculoskeletal symptoms predominantly affecting the neck, shoulders, and back, accompanied by localized troubles. Specifically, she reports bilateral issues, with left neck problems, left shoulder problems and trouble, and right shoulder problems and trouble, including difficulty performing actions like giving a thumbs up and wearing an armband on the right side. The patient is also experiencing lumbago, difficulties with the left hip joint, pain in the right shin, left hamstring discomfort, issues with the right tear ducts, right rear thigh pain, and a tendency for problems with the right Achilles tendon.

---



---


Patient ID: Neuro_00044
Age: 33
Gender: Female

Summary of Symptom Diagnosis:
The patien

In [105]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00091
Age: 35
Gender: Female

---


Patient ID: Neuro_00009
Age: 39
Gender: Female

Symptom Diagnosis:
The patient presents with a complex symptomatology including neck pain localized on the right side, issues with both shoulders, trouble with the right elbow, and problems with both hands, with specific mention of the right armband area. Additionally, the patient suffers from problems associated with the right leg, left pes anserinus (L pta), bilateral front knee pain, left ankle trouble, right heel concerns, pain in the right posterior thigh, and left Achilles tendon pain (achillodyni). The patient also reports upper abdominal discomfort.

---


Patient ID: Neuro_00095
Age: 21
Gender: Male

Symptom Diagnosis:
The patient presents with right (R) posterior tibial arteries (pta) issues, R shoulder p

In [106]:
example = ''

chosen_idx = g==g_chosen[3]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00008
Age: 24
Gender: Female

Symptom Diagnosis:
The patient presents with various musculoskeletal complaints including right (R) wrist and medial elbow problems. Additionally, there is a report of pain in the left (L) hamstring, right (R) posterior thigh, and left (L) dorsal knee joint disorders, suggesting multiple areas of involvement. The patient also expresses conditions related to both the left (L) and right (R) Achilles, with the right side displaying a tendency for issues. Notably, there is mention of 'right obesity', which is not standard nomenclature and requires further clarification.

---


Patient ID: Neuro_00041
Age: 43
Gender: Male

Symptom Diagnosis:
The patient presents with lumbago characterized by lower back pain, as well as radiating pain to the left lower extremity (L pta), pa

In [82]:
input_factors = '''  
0.
DLI C3-C4
Radiculopathy C5-C6

Location of symptoms on the dermatome map, specifically:
DLI C1-C2
Radiculopathy C7-T1
Type of symptoms reported, specifically:
Neck problems
Shoulder impingement

DLI C1-C2
Radiculopathy C5-C6

1.
DLI (C1-C2) (Template 1)
Radiculopathy (C3-C4) (Template 2)

DLI in the right shoulder (Template 1: "DLI [C5-C6]")
Radiculopathy in the right shoulder (Template 2: "[R] Radiculopathy [C5-C6]")

DLI (location on the dermatome map): Patients with DLI on the C4-C5 dermatome map.
Radiculopathy (location on the dermatome map): Patients with radiculopathy on the C5-C6 dermatome ma


2.
DLI C2
Radiculopathy L5
DLI C4
Radiculopathy L3

DLI C3-C4
L Radiculopathy C5-C6

DLI (Discoligamentous Injury) of the neck and shoulder region, specifically in the C3-C4 and C4-C5 dermatomes.
Radiculopathy in the left (L) and right (R) shoulder and arm regions, specifically in the C5-C6 and C6-C7 dermatomes.


3.
DLI C1-C2
L Radiculopathy T2-T6
R Radiculopathy T7-T12
DLI C3-C4
R Shoulder Impingement

DLI C2-C3: Discoligamentous injury in the C2-C3 dermatome map region.
L Radiculopathy C5-C6: Left radiculopathy in the C5-C6 dermatome map region.
R Radiculopathy C7-T1: Right radiculopathy in the C7-T1 dermatome map region.


DLI C4-C5: This factor captures the symptoms reported in the shoulder and upper arm region, which are commonly associated with DLI.
Radiculopathy T2-T6: This factor includes symptoms in the chest and abdomen region, which are often related to radiculopathy.
DLI C6-C7: This factor represents symptoms in the forearm and middle finger region, which can also be caused by DLI.
Radiculopathy L4-L5: This factor includes symptoms in the front and inner side of the leg, which are commonly associated with radiculopathy.

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['L T10 Radiculopathy',
 'L L4 Radiculopathy',
 'DLI C2-C3',
 'R T7 Radiculopathy',
 'L T6 Radiculopathy',
 'R L4 Radiculopathy',
 'R L3 Radiculopathy',
 'R T10 Radiculopathy']

### confirmation

In [83]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

L C6 Radiculopathy
R C6 Radiculopathy
L C5 Radiculopathy
R C5 Radiculopathy
DLI C5-C6
DLI C3-C4
R C4 Radiculopathy
L C7 Radiculopathy
R C7 Radiculopathy
DLI C4-C5
True


In [84]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 10: 100%|██████████| 11/11 [00:00<00:00, 540.88it/s]

[0.36319974 0.46921774 0.4166811  0.40270162 0.4356054  0.38644277
 0.4        0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="L C7 Radiculopathy"];
4 [label=X5];
4 [label="R C6 Radiculopathy"];
5 [label=X6];
5 [label="R C7 Radiculopathy"];
6 [label=X7];
6 [label="DLI C3-C4"];
7 [label=X8];
7 [label="L C6 Radiculopathy"];
8 [label=X9];
8 [label="L C5 Radiculopathy"];
9 [label=X10];
9 [label="DLI C4-C5"];
10 [label=X11];
10 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
9 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 5  [arrowhead=odot, arrowtail=odo

In [85]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'DLI C3-C4', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	6	2
1.0	0.9743589743589743	0.8571428571428572	0.75


## Llama-2-13b Zeroshot

https://poe.com/s/SFTKOI7ChR8PbOm1Bp2w

In [86]:
input_factors = '''  
DLI C4-C5: Discoligamentous injury in the C4-C5 region of the cervical spine.
Radiculopathy L4-L5: Compression of the L4-L5 nerve roots.


DLI C5: This factor involves a discoligamentous injury on the C5 dermatome map, which is located lateral to the upper arm and elbow.
Radiculopathy C5: This factor involves nerve compression or irritation of the C5 nerve, which is located on the C5 dermatome map.

DLI C4-C5: Discoligamentous injury in the C4-C5 region of the cervical spine.
L Radiculopathy C5-C6: Radiculopathy in the C5-C6 region of the cervical spine on the left side.

'''.split('\n')
proposed_zero_shot_factors = get_variables_with_data(input_factors)
proposed_zero_shot_factors

['R L5 Radiculopathy',
 'DLI C5-C6',
 'L L4 Radiculopathy',
 'R L4 Radiculopathy',
 'L C6 Radiculopathy',
 'L C5 Radiculopathy',
 'DLI C4-C5',
 'L L5 Radiculopathy',
 'R C5 Radiculopathy']

In [87]:
print(*Eval_Factor_Discovery(proposed_zero_shot_factors), sep='\t')
print(*Eval_Factor_Confirmation(proposed_zero_shot_factors, proposed_zero_shot_factors, mode = 'ancestors'), sep='\t')

1	3	6
0.5	0.8846153846153846	0.4	0.3333333333333333


# Llama-2-7b

https://poe.com/Llama-2-7b

https://poe.com/s/YoLMG4TpcjdugHeORPlT

In [88]:
V = set(['R shoulder impingement'])
all_factors = set(['R shoulder impingement'])
factors = [ f for f in V if f!= target_name]
factors

[]

## iteration 1

In [114]:
print(propose_prompt_with_data_in_front(init_data))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data

 

### Group with 'R shoulder impingement'=0

----


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impingement, accompanied by non-specific problems with the right shoulder. The patient also reports issues in the lower body, including concerns with the left adductor tendon, unspecified lumbago, discomfort in the left hip joint, and pain in the right shin. Further symptoms involve trouble with the left knee, pain in the left hamstring, issues with the left toe, and pain located in the left rear thigh.

----


Patient ID: Neuro_00040
Age: 19
Gender: Male

Symptom Diagnosis:
The patient presents with a range of symptoms predominantly relating to the lower back and left lowe

### feedback

In [89]:

input_factors = '''  

Chosen factors: DLI (location on the dermatome map) and [L/R] Radiculopathy (location on the dermatome map)
Interpreted the data based on the chosen factors:
Group 1: Patients in this group present with symptoms related to the left shoulder impingement, which may be related to the C5 dermatome.
Group 2: Patients in this group present with symptoms related to the right shoulder impingement, which may be related to the C6 dermatome.
Group 3: Patients in this group present with a mix of symptoms related to both the left and right shoulders, which may be related to the C5 and C6 dermatomes, respectively.
The final output is as follows:
Group 1:
DLI: C5
Radiculopathy: C6
Group 2:
DLI: C6
Radiculopathy: C5
Group 3:
DLI: Both C5 and C6
Radiculopathy: Both C5 and C6


DLI (location on the dermatome map)
[L/R] Radiculopathy (location on the dermatome map)

DLI C1-C2: Discoligamentous injury located on the C1 and C2 dermatomes.
DLI C3-C4: Discoligamentous injury located on the C3 and C4 dermatomes.
DLI C5-C6: Discoligamentous injury located on the C5 and C6 dermatomes.
DLI C7-C8: Discoligamentous injury located on the C7 and C8 dermatomes.
[R] Radiculopathy C1-C2: Radiculopathy located on the C1 and C2 dermatomes.
[R] Radiculopathy C3-C4: Radiculopathy located on the C3 and C4 dermatomes.
[R] Radiculopathy C5-C6: Radiculopathy located on the C5 and C6 dermatomes.
[R] Radiculopathy C7-C8: Radiculopathy located on the C7 and C8 dermatomes.

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['R C8 Radiculopathy',
 'DLI C1-C2',
 'DLI C5-C6',
 'R C4 Radiculopathy',
 'R C6 Radiculopathy',
 'R C2 Radiculopathy',
 'DLI C7-C8',
 'DLI C3-C4',
 'R C7 Radiculopathy',
 'DLI C6-C7',
 'R C3 Radiculopathy',
 'L C6 Radiculopathy',
 'L C5 Radiculopathy',
 'DLI C2-C3',
 'DLI C4-C5',
 'R C5 Radiculopathy']

### confirmation

In [90]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

DLI C5-C6
R C4 Radiculopathy
R C6 Radiculopathy
DLI C3-C4
R C7 Radiculopathy
R C5 Radiculopathy
L C6 Radiculopathy
L C5 Radiculopathy
DLI C4-C5
False


In [91]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 9: 100%|██████████| 10/10 [00:00<00:00, 657.29it/s]

[0.36319974 0.46921774 0.4166811  0.4356054  0.38644277 0.4
 0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="R C6 Radiculopathy"];
4 [label=X5];
4 [label="R C7 Radiculopathy"];
5 [label=X6];
5 [label="DLI C3-C4"];
6 [label=X7];
6 [label="L C6 Radiculopathy"];
7 [label=X8];
7 [label="L C5 Radiculopathy"];
8 [label=X9];
8 [label="DLI C4-C5"];
9 [label=X10];
9 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
8 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 3  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];

In [92]:
# meta_data
print(*Eval_Factor_Discovery(factors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, factors, mode = 'ancestors'), sep='\t')

3	6	3
1.0	0.9615384615384616	0.8	0.6666666666666666


In [94]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'R C7 Radiculopathy', 'DLI C3-C4', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	6	3
1.0	0.9615384615384616	0.8	0.6666666666666666


## iteration 2

### feedback

In [95]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.10656595882801999 45 [0.97777778 0.02222222]
1 0.5929533174474745 25 [0.72 0.28]
2 0.6474466390346325 20 [0.65 0.35]
3 0.6931471805599453 10 [0.5 0.5]
choose g==[0, 1, 2, 3]


In [120]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00065
Age: 26
Gender: Male

Symptom Diagnosis Summary:
The patient presents with an array of musculoskeletal symptoms affecting multiple areas of the body. The patient reports experiencing problems specifically with the left neck and right neck. In addition, there is impingement and trouble associated with both the left and right shoulders. The patient describes difficulties with his left arm and left hand, as well as medial elbow problems on both sides. Moreover, the patient is experiencing lumbago, pain attributed to the left patella tendon (L pta), and right hip arthritis. The patient also mentions having issues with his left foot functionality. Notably, the patient has reported left-sided obesity, which may be a contributing factor or consequence of his musculoskeletal complaints.

---


Patie

In [121]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00000
Age: 57
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a left shoulder impingement and associated intracapsular problems. The symptoms include limited and painful range of motion with a positive outcome when performing a "thumbs up" test on the left side. Right upper limb assessment also reveals issues; the patient wears an armband suggesting ongoing management for right lower arm disorders. The patient reports lumbago and pain localized to the left thigh. Physical therapy assessment (L pta) is noted, indicating involvement in a therapeutic intervention. Joint assessments reveal a left hip joint condition and right hip arthritis, accompanied by right hamstring discomfort.

---


Patient ID: Neuro_00011
Age: 56
Gender: Male

Summary of Symptom Diagnosis:
The patient presen

In [122]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00017
Age: 38
Gender: Male

Symptom Diagnosis Summary:
The patient presents with lumbago, characterized by pain in the lower back area. Additionally, the patient is experiencing pain specifically on the left side of the abdomen, indicative of left-sided obesity. He is reporting right hamstring (R ham) pain and pain in the left rear thigh. These symptoms suggest possible musculoskeletal or nerve-related issues in the lumbar region affecting various nearby structures and muscles.

---


Patient ID: Neuro_00057
Age: 17
Gender: Female

Symptom Diagnosis Summary:
The patient presents with a range of symptoms suggestive of cervical origin, including neck pain that is localized on the right side. There is also a complaint of headache, specifically affecting the left frontal region, which may be indicativ

In [123]:
example = ''

chosen_idx = g==g_chosen[3]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00071
Age: 44
Gender: Male

Symptom Diagnosis:
The patient presents with a constellation of symptoms that include problems with the left neck, right shoulder, and right ham coupled with right shoulder trouble with specific difficulty in performing a 'thumbs up.' The patient also reports trouble in the left hip joint, lower left leg, and left ankle, in addition to right obesity and upper abdominal discomfort.

---


Patient ID: Neuro_00082
Age: 57
Gender: Female

Symptom Diagnosis Summary:
The patient presents with unilateral neck discomfort on the right side, associated with impingement and trouble in the left shoulder, as well as unspecified problems in the right shoulder, contributing to difficulties in upper limb function. There is also a reported discomfort in the left upper arm and problems a

In [96]:
input_factors = '''  
0.
Group 1: Patients with DLI C1-C2 or [L/R] Radiculopathy C1-C2.
Group 2: Patients with DLI C3-C4 or [L/R] Radiculopathy C3-C4.
Group 3: Patients with DLI C5-C6 or [L/R] Radiculopathy C5-C6.
Group 4: Patients with DLI C7-C8 or [L/R] Radiculopathy C7-C8.

Using the factors proposed in Part 1, the possible allocation of groups is as follows:

Group 1: DLI C1-C2 (top of the head, back of the head)
Patient ID: Neuro_00065
Symptom Diagnosis Summary: The patient presents with left neck problems and right partial thromboplastin time (R pta) issues, which may be related to sensory distribution in the C1-C2 dermatome areas.
Group 2: [L/R] Radiculopathy T1-T6 (chest area)
Patient ID: Neuro_00093
Symptom Diagnosis Summary: The patient presents with left shoulder impingement and right partial thromboplastin time (R pta) issues, which may be related to sensory distribution in the T1-T6 dermatome areas.
Group 3: Lumbar Dermatomes (L1-L5)
Patient ID: Neuro_00028
Symptom Diagnosis Summary: The patient presents with lumbago, pain attributed to the left patella tendon (L pta), and right hip arthritis, which may be related to sensory distribution in the L1-L5 dermatome areas.
Group 4: [L/R] Radiculopathy S1-S5 (back of the leg and the heel, buttocks and perineal area, sacrococcygeal area)
Patient ID: Neuro_00054
Symptom Diagnosis Summary: The patient presents with left-sided obesity,

DLI (Location on the Dermatome Map):
DLI C1-C2: Discoligamentous injury affecting the top and back of the head.
DLI C3-C4: Discoligamentous injury affecting the shoulder area and the back of the neck.
DLI C5-C6: Discoligamentous injury affecting the arm and the thumb side of the hand.
DLI C7-C8: Discoligamentous injury affecting the forearm and the little finger side of the hand.
[L/R] Radiculopathy (Location on the Dermatome Map):
[L] Radiculopathy C1-C2: Radiculopathy affecting the top and back of the head.
[L] Radiculopathy C3-C4: Radiculopathy affecting the shoulder area and the back of the neck.
[R] Radiculopathy C5-C6: Radiculopathy affecting the arm and the thumb side of the hand.
[R] Radiculopathy C7-C8: Radiculopathy affecting the forearm and the little finger side of the hand.


1.

DLI (location on the dermatome map)
[L/R] Radiculopathy (location on the dermatome map)

Based on my consideration, I have decided to use the following factors for allocation:
DLI (location on the dermatome map)
C1: Top of the head
C2: Back of the head
C3: Back of the neck
C4: Shoulder area
C5: Lateral upper arm to the elbow
C6: Forearm and the thumb side of the hand
C7: Forearm and the middle finger
C8: Little finger side of the hand
[L/R] Radiculopathy (location on the dermatome map)
L1: Groin area
L2: Front of the thigh
L3: Knee region
L4: Front and inner side of the leg
L5: Outer side of the leg and the top of the foot
The patients can be allocated into the following groups based on these factors:
Group 1: Patient ID 00000
DLI: C1, C2, C3, C4, C5, C6, C7, C8
Radiculopathy: L1, L2, L3, L4, L5
Group 2: Patient ID 00011
DLI: C1, C2, C3, C4, C5, C6, C7, C8
Radiculopathy: L1, L2, L3, L4, L5
Group 3: Patient ID 00009
DLI: C1, C2, C3, C4, C5, C6, C7, C8
Radiculopathy: L1, L2, L3, L4, L5
Group 4: Patient ID 00058
DLI: C1, C2, C3, C4, C5, C6, C7, C8
Radiculopathy: L1, L2, L3, L4, L5

DLI [location on the dermatome map]
[L/R] Radiculopathy [location on the dermatome map]

2.

DLI C1-C2: This factor considers the discoligamentous injuries in the C1 and C2 dermatome areas.
Radiculopathy C3-C4: This factor considers the radiculopathy in the C3 and C4 dermatome areas.

DLI (location on the dermatome map)
[L/R] Radiculopathy (location on the dermatome map)

The factors I decided to use are:
DLI (location on the dermatome map)
[L/R] Radiculopathy (location on the dermatome map)
The output for each group is as follows:
Group 0:
DLI: C1-C2
Radi: None
Group 1:
DLI: C2
Radi: C2
Group 2:
DLI: C4
Radi: C4
Group 3:
DLI: C7
Radi: C7
Group 4:
DLI: L5
Radi: L5

3.

DLI C1-C2
Radi C1-C2

DLI C1-C2: Top of the head, back of the head
DLI C3-C4: Back of the neck, shoulder area
DLI C5-C6: Forearm, thumb side of the hand
DLI C7-C8: Forearm, middle finger, little finger side of the hand
[L/R] Radiculopathy L: Left side
[L/R] Radiculopathy R: Right side

DLI (C1-C2): Discoligamentous injuries in the cervical spine are associated with neck pain and difficulty with shoulder function.
[L/R] Radiculopathy (C1-C2): Radiculopathy in the cervical spine is associated with symptoms affecting upper limb function, including trouble with thumb movements and pain in the shoulder and arm.

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['L S1 Radiculopathy',
 'L T10 Radiculopathy',
 'L L4 Radiculopathy',
 'R L1 Radiculopathy',
 'L L1 Radiculopathy',
 'L T6 Radiculopathy',
 'R L4 Radiculopathy',
 'L L3 Radiculopathy',
 'DLI L5-S1',
 'R L3 Radiculopathy',
 'R T10 Radiculopathy',
 'L C4 Radiculopathy',
 'R S1 Radiculopathy',
 'L C8 Radiculopathy',
 'R L5 Radiculopathy',
 'L C7 Radiculopathy',
 'L C3 Radiculopathy',
 'L C2 Radiculopathy',
 'L L5 Radiculopathy']

### confirmation

In [97]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

L C6 Radiculopathy
R C6 Radiculopathy
L C5 Radiculopathy
R C5 Radiculopathy
DLI C5-C6
DLI C3-C4
R C4 Radiculopathy
L C7 Radiculopathy
R C7 Radiculopathy
DLI C4-C5
False


In [98]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 10: 100%|██████████| 11/11 [00:00<00:00, 453.75it/s]

[0.36319974 0.46921774 0.4166811  0.40270162 0.4356054  0.38644277
 0.4        0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="L C7 Radiculopathy"];
4 [label=X5];
4 [label="R C6 Radiculopathy"];
5 [label=X6];
5 [label="R C7 Radiculopathy"];
6 [label=X7];
6 [label="DLI C3-C4"];
7 [label=X8];
7 [label="L C6 Radiculopathy"];
8 [label=X9];
8 [label="L C5 Radiculopathy"];
9 [label=X10];
9 [label="DLI C4-C5"];
10 [label=X11];
10 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
9 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 5  [arrowhead=odot, arrowtail=odo

In [99]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'DLI C3-C4', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	6	2
1.0	0.9743589743589743	0.8571428571428572	0.75


## iteration 3

### feedback

In [100]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.410116318288409 14 [0.85714286 0.14285714]
1 0.666278442414676 13 [0.61538462 0.38461538]
2 0.6108643020548935 30 [0.7 0.3]
3 0.3094772867741259 43 [0.90697674 0.09302326]
choose g==[0, 1, 2, 3]


In [133]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impingement, accompanied by non-specific problems with the right shoulder. The patient also reports issues in the lower body, including concerns with the left adductor tendon, unspecified lumbago, discomfort in the left hip joint, and pain in the right shin. Further symptoms involve trouble with the left knee, pain in the left hamstring, issues with the left toe, and pain located in the left rear thigh.

---


Patient ID: Neuro_00028
Age: 55
Gender: Male

SYMPTOM DIAGNOSIS:
The patient presents with a multifocal array of symptoms including headaches localized to the forehead

In [134]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient id: Neuro_00068
Age: 42
Gender: Male

Symptom diagnosis:
The patient presents with a range of musculoskeletal symptoms including left neck problems, complications with both shoulders, and additional trouble in the left and right hands. The patient describes specific difficulties with the left small finger, alongside issues with both medial elbows. These varied symptoms are suggestive of a multifocal neurological or musculoskeletal condition affecting multiple limbs and warrant thorough investigation.

---


Patient ID: Neuro_00044
Age: 33
Gender: Female

Summary of Symptom Diagnosis:
The patient has presented with pain localized to the neck and has been experiencing issues with both shoulders. Specifically, there is documentation of problems in the left shoulder as well as trouble in the right shoulder. The

In [135]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00046
Age: 27
Gender: Female

Symptom Diagnosis Summary:
The patient presents with musculoskeletal complaints encompassing the cervical, shoulder, lumbar, and lower extremity regions. Specifically, there is neck pain localized to the right neck, indicating a possible unilateral cervical issue. The left shoulder exhibits impingement and other unspecified problems, while the right wrist and hand are also symptomatic. Complaints include pain in the left adductor tendon and bilateral lumbago, with a particular emphasis on the right side. The patient additionally reports obesity, which might be contributing to the lower limb issues, like the left dorsal knee joint disorder, right heel discomfort, and complications with the left Achilles tendon. 

---


Patient ID: Neuro_00088
Age: 52
Gender: Male

Summ

In [136]:
example = ''

chosen_idx = g==g_chosen[3]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient id: Neuro_00006
Age: 19
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a range of symptoms including left-sided tinnitus, headache, and jaw problems. The patient is also experiencing pharyngeal discomfort and left backache. There is noted right primary trochlear apparatus (R pta) dysfunction, as well as left-sided primary trochlear apparatus (L pta) issues. The patient reports right hip arthritis and a medial disorder of the left knee joint, in addition to pain in the left shin. The patient is also described as having right-sided obesity-related issues, pain in the calcaneal region, and right achillodynia.

---


Patient ID: Neuro_00021
Age: 48
Gender: Male

Symptom Diagnosis:
The patient presents with a variety of musculoskeletal complaints including lumbago, or lower back pain, affecti

In [101]:
input_factors = '''  
0.
Based on the considerations and hypothesis, the factors that I will use to allocate the samples to the groups are:
Group 0: DLI C1-C2 and L Radiculopathy C1-C2
The patients in this group have a discoligamentous injury affecting the upper cervical region, leading to radiculopathy in the lower cervical nerves.
Group 1: DLI C3-C4 and L Radiculopathy C3-C4
The patients in this group have a discoligamentous injury affecting the mid-cervical region, leading to radiculopathy in the mid-cervical nerves.
Group 2: DLI C5-C6 and L Radiculopathy C5-C6
The patients in this group have a discoligamentous injury affecting the upper cervical region, leading to radiculopathy in the upper cervical nerves.
Group 3: DLI C7-C8 and L Radiculopathy C7-C8
The patients in this group have a discoligamentous injury affecting the thoracic region, leading to radiculopathy in the thoracic nerves.
The patients in each group have a specific underlying cause of their symptoms, which can be addressed through targeted treatment.


Group 1: DLI C1-C2
Patient ID: Neuro_00083
Symptom Diagnosis: Left neck problems, left shoulder impingement, non-specific problems with the right shoulder, and pain in the left hip joint.
DLI: C1-C2
Group 2: [L] Radiculopathy C5-C6
Patient ID: Neuro_00054
Symptom Diagnosis: Multi-focal musculoskeletal complaints primarily affecting the neck, shoulders, lower back, and right lower extremity.
[L] Radiculopathy: C5-C6
Group 3: DLI L5
Patient ID: Neuro_00090
Symptom Diagnosis: Right patellar tendinitis (R pta), bilateral shoulder impairment with evident impingement on the left, and generalized shoulder problems on both sides.
DLI: L5
Group 4: [R] Radiculopathy L4-L5
Patient ID: Neuro_00070
Symptom Diagnosis: Right-sided upper extremity issues, including pain in the right neck, impingement in the right shoulder, trouble with both the right shoulder and right elbow, and wearing an armband on the right arm.
[R] Radiculopathy: L4-L5


DLI (location on the dermatome map):
C1: Top of the head
C2: Back of the head
C3: Back of the neck
C4: Shoulder area
C5: Lateral upper arm to the elbow
C6: Forearm and the thumb side of the hand
C7: Forearm and the middle finger
C8: Little finger side of the hand
[L/R] Radiculopathy (location on the dermatome map):
L1: Groin area
L2: Front of the thigh
L3: Knee region
L4: Front and inner side of the leg
L5: Outer side of the leg and the top of the foot

1.
DLI [location on the dermatome map]:
Patient ID: Neuro_00068
DLI location: C5 (thumb side of the hand)
Patient ID: Neuro_00044
DLI location: C6 (middle finger)
Patient ID: Neuro_00066
DLI location: C4 (shoulder area)
[L/R] Radiculopathy [location on the dermatome map]:
Patient ID: Neuro_00039
Radiculopathy location: C8 (little finger side of the hand)
Patient ID: Neuro_00056
Radiculopathy location: C7 (middle finger)

DLI C1-C2: Discoligamentous injury affecting the C1 and C2 dermatome regions.
Radi C1-C2: Radiculopathy affecting the C1 and C2 dermatome regions.

DLI (location on the dermatome map):
DLI C1-C2: Discoligamentous injury affecting the nerves supply to the scalp, face, and neck.
DLI C3-C4: Discoligamentous injury affecting the nerves supply to the shoulder, arm, and hand.
DLI C5-C6: Discoligamentous injury affecting the nerves supply to the forearm, wrist, and hand.
DLI C7-C8: Discoligamentous injury affecting the nerves supply to the upper limb, including the arm and hand.
[L/R] Radiculopathy (location on the dermatome map):
Radi C1-C2: Radiculopathy affecting the nerves supply to the scalp, face, and neck.
Radi C3-C4: Radiculopathy affecting the nerves supply to the shoulder, arm, and hand.
Radi C5-C6: Radiculopathy affecting the nerves supply to the forearm, wrist, and hand.
Radi C7-C8: Radiculopathy affecting the nerves supply to the upper limb, including the arm and hand.

2.
DLI [location on the dermatome map]
[L/R] Radiculopathy [location on the dermatome map]

DLI (location on the dermatome map):
DLI C1-C2: C1 and C2 dermatomes (top and back of the head)
DLI C3-C4: C3 and C4 dermatomes (back of the neck and shoulder area)
DLI C5-C6: C5 and C6 dermatomes (forearm and thumb side of the hand)
DLI C7-C8: C7 and C8 dermatomes (forearm and little finger side of the hand)
[L/R] Radiculopathy (location on the dermatome map):
L Radiculopathy C1-C2: Left radiculopathy in C1 and C2 dermatomes (top and back of the head)
L Radiculopathy C3-C4: Left radiculopathy in C3 and C4 dermatomes (back of the neck and shoulder area)
L Radiculopathy C5-C6: Left radiculopathy in C5 and C6 dermatomes (forearm and thumb side of the hand)
L Radiculopathy C7-C8: Left radiculopathy in C7 and C8 dermatomes (forearm and little finger side of the hand)
R Radiculopathy C1-C2: Right radiculopathy in C1 and C2 dermatomes (top and back of the head)
R Radiculopathy C3-C4: Right radiculopathy

DLI C1-C2: There is a discoligamentous injury in the C1 and C2 dermatomes, which are located in the neck region. This could contribute to shoulder impingement.
Radi C1-C2: There is radiculopathy in the C1 and C2 dermatomes, which are located in the neck region. This could also contribute to shoulder impingement.


3.
Location of symptoms (e.g., left/right, front/back, neck, jaw, pharyngeal, etc.)
Presence of radiculopathy or DLI (discoligamentous injury) in specific locations on the dermatome map

DLI (location on the dermatome map)
DLI C1-C2: Discoligamentous injury affecting the C1 and C2 dermatomes, which are located on the top and back of the head, respectively.
DLI C3-C4: Discoligamentous injury affecting the C3 and C4 dermatomes, which are located on the back of the neck and shoulder area, respectively.
DLI C5-C8: Discoligamentous injury affecting the C5 and C8 dermatomes, which are located on the forearm and hand, respectively.
[L/R] Radiculopathy (location on the dermatome map)
Radi C1-C2: Radiculopathy affecting the C1 and C2 dermatomes, which are located on the top and back of the head, respectively.
Radi C3-C4: Radiculopathy affecting the C3 and C4 dermatomes, which are located on the back of the neck and shoulder area, respectively.
Radi C5-C8: Radiculopathy affecting the C5 and C8 dermatomes, which are located on the forearm and hand, respectively.
Obesity-related issues
Obesity: Obesity-related issues may contribute to musculoskeletal problems and neurological symptoms, such as headache, tinnitus, and pharyngeal discomfort.
Musculoskeletal problems
Musculoskeletal pain: Musculoskeletal pain may be associated with a variety of factors, including degenerative joint disease, injury, and overuse.
Musculoskeletal inflammation: Musculoskeletal inflammation may be associated with a variety of factors, including infection, autoimmune disorders, and inflammatory conditions.
Neurological symptoms
Headache: Headache may be associated with a variety of factors, including musculoskeletal problems, neurological disorders, and other conditions.
Tinnitus: Tinnitus may be associated with a variety of factors, including ear infections, earwax buildup, and neurological disorders.
Pharyngeal discomfort: Pharyngeal discomfort may be associated with a variety of factors, including respiratory infections, allergies, and neurological disorders.


Factors to consider for allocation of groups:
DLI (location on the dermatome map)
[L/R] Radiculopathy (location on the dermatome map)




'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

[]

### confirmation

In [102]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

L C6 Radiculopathy
R C6 Radiculopathy
L C5 Radiculopathy
R C5 Radiculopathy
DLI C5-C6
DLI C3-C4
R C4 Radiculopathy
L C7 Radiculopathy
R C7 Radiculopathy
DLI C4-C5
True


In [103]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 10: 100%|██████████| 11/11 [00:00<00:00, 512.79it/s]

[0.36319974 0.46921774 0.4166811  0.40270162 0.4356054  0.38644277
 0.4        0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="L C7 Radiculopathy"];
4 [label=X5];
4 [label="R C6 Radiculopathy"];
5 [label=X6];
5 [label="R C7 Radiculopathy"];
6 [label=X7];
6 [label="DLI C3-C4"];
7 [label=X8];
7 [label="L C6 Radiculopathy"];
8 [label=X9];
8 [label="L C5 Radiculopathy"];
9 [label=X10];
9 [label="DLI C4-C5"];
10 [label=X11];
10 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
9 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 5  [arrowhead=odot, arrowtail=odo

In [104]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'DLI C3-C4', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	6	2
1.0	0.9743589743589743	0.8571428571428572	0.75


## Llama-2-7b Zeroshot

https://poe.com/s/FLc9ngloh7J56rFRN90G

In [105]:
input_factors = '''  
DLI C1-C2
[L/R] Radiculopathy C3-C5
DLI C6-C7
[L/R] Radiculopathy T1-T4

DLI (C1-C2) on the dermatome map.
Radiculopathy (R) (C5-C8) on the dermatome map.
DLI (C3) on the dermatome map.

I propose the following factors as potential causes of right shoulder impingement:

Dislocation or subluxation of the cervical spine, particularly in the C1-C2 or C2-C3 joints.
DLI (dermatome location) of the C1-C2 dermatome, which could result in compression of the brachial plexus nerve roots.
Radiculopathy in the C1-C2 dermatome, which could cause compression of the brachial plexus nerve roots and result in right shoulder impingement.
Muscle imbalances or weakness in the shoulder muscles, leading to poor posture or abnormal movement patterns that put pressure on the right shoulder.
Poorly managed or untreated conditions such as cervical spine stenosis or thoracic outlet syndrome, which could cause nerve compression and lead to right shoulder impingement.


'''.split('\n')
proposed_zero_shot_factors = get_variables_with_data(input_factors)
proposed_zero_shot_factors

['R T10 Radiculopathy',
 'R C8 Radiculopathy',
 'DLI C1-C2',
 'L T10 Radiculopathy',
 'R T4 Radiculopathy',
 'L C3 Radiculopathy',
 'L T11 Radiculopathy',
 'R T11 Radiculopathy',
 'L T4 Radiculopathy',
 'R C2 Radiculopathy',
 'R T12 Radiculopathy',
 'DLI C6-C7',
 'L T1 Radiculopathy',
 'R C3 Radiculopathy',
 'L T12 Radiculopathy',
 'L C5 Radiculopathy',
 'R T1 Radiculopathy',
 'R C5 Radiculopathy']

In [106]:
print(*Eval_Factor_Discovery(proposed_zero_shot_factors), sep='\t')
print(*Eval_Factor_Confirmation(proposed_zero_shot_factors, proposed_zero_shot_factors, mode = 'ancestors'), sep='\t')

1	1	17
0.16666666666666666	0.717948717948718	0.08333333333333333	0.05555555555555555


# GPT-4-Turbo

https://poe.com/GPT-4

https://poe.com/s/52duXyA2AkMDyHeqRwht

In [175]:
V = set(['R shoulder impingement'])
all_factors = set(['R shoulder impingement'])
factors = [ f for f in V if f!= target_name]
factors

[]

## iteration 1

In [176]:
print(propose_prompt_with_data_in_front(init_data))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data

 

### Group with 'R shoulder impingement'=0

----


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impingement, accompanied by non-specific problems with the right shoulder. The patient also reports issues in the lower body, including concerns with the left adductor tendon, unspecified lumbago, discomfort in the left hip joint, and pain in the right shin. Further symptoms involve trouble with the left knee, pain in the left hamstring, issues with the left toe, and pain located in the left rear thigh.

----


Patient ID: Neuro_00040
Age: 19
Gender: Male

Symptom Diagnosis:
The patient presents with a range of symptoms predominantly relating to the lower back and left lowe

### feedback

In [177]:

input_factors = '''  
"R Radiculopathy C5-C6" for the group with 'R shoulder impingement'=1.

Based on the analysis above, the factors for each group can be proposed as follows:
Group with 'R shoulder impingement'=0:
No factors involving right shoulder radiculopathy are identified as none of the patients specifically mention right shoulder impingement.
Group with 'R shoulder impingement'=1:
"R Radiculopathy C5-C6": This is based on the explicit mention of right shoulder impingement in this group, which corresponds to the C5-C6 dermatomes.

For Group with 'R shoulder impingement'=1:
"R Radiculopathy C5" - This dermatome is related to shoulder area and may explain the right shoulder impingement.
"R Radiculopathy C6" - This dermatome affects the thumb side of the hand, which may be linked with the upper extremity issues, although it's less specific to shoulder impingement.
For Group with 'R shoulder impingement'=0:
No specific factors can be confidently proposed based on the provided data and template constraints. The symptoms are diverse and do not point directly to DLI or radiculopathy in a specific dermatome region without further information or diagnostic results.


'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['R C6 Radiculopathy', 'R C5 Radiculopathy']

### confirmation

In [178]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

R C6 Radiculopathy
R C5 Radiculopathy
False


In [179]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 2: 100%|██████████| 3/3 [00:00<00:00, 1159.39it/s]

[0.36319974 0.4356054  0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="R C6 Radiculopathy"];
2 [label=X3];
2 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
}



In [174]:
# meta_data
print(*Eval_Factor_Discovery(factors), sep='\t')
print(*Eval_Factor_Confirmation(factors, factors, mode = 'ancestors'), sep='\t')

2	2	0
0.3333333333333333	0.9487179487179487	0.5	1.0


In [180]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['R C6 Radiculopathy', 'R C5 Radiculopathy']
2	2	0
0.3333333333333333	0.9487179487179487	0.5	1.0


## iteration 2

### feedback

In [181]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.5763341277567499 19 [0.73684211 0.26315789]
1 0.4791656239282754 81 [0.81481481 0.18518519]
choose g==[0, 1]


In [152]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00071
Age: 44
Gender: Male

Symptom Diagnosis:
The patient presents with a constellation of symptoms that include problems with the left neck, right shoulder, and right ham coupled with right shoulder trouble with specific difficulty in performing a 'thumbs up.' The patient also reports trouble in the left hip joint, lower left leg, and left ankle, in addition to right obesity and upper abdominal discomfort.

---


Patient ID: Neuro_00038

---


Patient ID: Neuro_00065
Age: 26
Gender: Male

Symptom Diagnosis Summary:
The patient presents with an array of musculoskeletal symptoms affecting multiple areas of the body. The patient reports experiencing problems specifically with the left neck and right neck. In addition, there is impingement and trouble associated with both the left and right shoulder

In [153]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00088
Age: 52
Gender: Male

Summary of Symptom Diagnosis:
The 52-year-old male patient presents with a complex array of musculoskeletal complaints involving multiple anatomical regions. Specifically, he reports left neck problems and pain in the right post-tibial area, accompanied by shoulder impingement and problems in both the left and right shoulders. The patient also describes dysfunction and discomfort in both hands, medial elbow problems on the left, and lumbago. In addition, he suffers from left post-tibial area discomfort, right hip arthritis, pain in the left anterior knee, right shin trouble, difficulties with the left footstool, and right hamstring issues. These symptoms suggest involvement of various musculoskeletal structures on both sides of the body.

---


Patient id: Neuro_00068
A

In [182]:
input_factors = '''  
0.
R Radiculopathy C4-C6: Patients in the 'R shoulder impingement' = 1 group report symptoms that suggest involvement of the C4 to C6 nerve roots on the right side.
DLI C4-C6: Although not explicitly mentioned, the symptoms of right shoulder impingement could also be a result of a discoligamentous injury in the cervical region that corresponds to the areas innervated by C4 to C6 nerve roots.

"R Radiculopathy C5": This is suggested due to the presence of right shoulder impingement in the patients from Group 1 and the lack of specific impingement in Group 0.
"R Radiculopathy C6": This is indicated due to the problems with the thumb (which is innervated by C6) and the right shoulder and lateral elbow pain reported by patients in Group 1.

"R Radiculopathy C4-C5": This factor is chosen because the right shoulder impingement is likely associated with issues in the nerves emanating from the cervical spine that innervate the shoulder area.
"R Radiculopathy C6": This factor could be considered given the presence of right arm (specifically thumb side) symptoms in patients with 'R shoulder impingement'.


1.
For group with 'R shoulder impingement' = 0 (no right shoulder impingement):
DLI C6-C7 (due to complaints involving the forearm, hands, and fingers which could be related to issues in these dermatomes but not specifically the shoulder impingement).
L Radiculopathy C5-C6 (due to left neck problems and hand issues without explicit mention of right shoulder impingement).
For group with 'R shoulder impingement' = 1 (presence of right shoulder impingement):
R Radiculopathy C4-C5 (due to direct mention of right shoulder impingement and troubles, which directly correlates with the shoulder dermatomes).
DLI C4-C5 (due to the general issues and pain in the right shoulder region that could be indicative of disc or ligamentous injuries at these levels).

"R Radiculopathy C5"
"R Radiculopathy C6"

Patient group with 'R shoulder impingement' = 0: No specific factors can be determined from the data provided for discoligamentous injury or radiculopathy using the allowed templates. The symptoms do not point explicitly to issues with a particular dermatome or suggest radiculopathy without more specific information.
Patient group with 'R shoulder impingement' = 1:
"R Radiculopathy C5" (due to right shoulder impingement and potential lateral arm pain)
"R Radiculopathy C6" (due to right shoulder impingement and involvement of the thumb side of the hand and forearm indicated for Patient ID: Neuro_00036)

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['DLI C5-C6',
 'R C4 Radiculopathy',
 'DLI C6-C7',
 'L C6 Radiculopathy',
 'L C5 Radiculopathy',
 'DLI C4-C5']

### confirmation

In [183]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

DLI C5-C6
R C4 Radiculopathy
R C6 Radiculopathy
L C6 Radiculopathy
L C5 Radiculopathy
DLI C4-C5
R C5 Radiculopathy
False


In [184]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 7: 100%|██████████| 8/8 [00:00<00:00, 800.99it/s]

[0.36319974 0.46921774 0.4166811  0.4356054  0.44301279 0.34220843
 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="R C6 Radiculopathy"];
4 [label=X5];
4 [label="L C6 Radiculopathy"];
5 [label=X6];
5 [label="L C5 Radiculopathy"];
6 [label=X7];
6 [label="DLI C4-C5"];
7 [label=X8];
7 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
6 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 3  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
5 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
5 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
6 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [185]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	5	2
0.8333333333333334	0.9615384615384616	0.7692307692307694	0.7142857142857143


## iteration 3

### feedback

In [186]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.410116318288409 14 [0.85714286 0.14285714]
1 0.45056120886630463 12 [0.83333333 0.16666667]
2 0.6730116670092565 25 [0.6 0.4]
3 0.37177617994345286 49 [0.87755102 0.12244898]
choose g==[0, 1, 2, 3]


In [148]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00061
Age: 29
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a complex array of symptoms which include pain localized to the right neck and bilateral shoulder issues characterized by impingement and generalized trouble, more so on the left side. Additionally, the patient reports experiencing difficulties with the right thumb (associated with a positive "thumbs up" sign), left hand troubles suggestive of functional impairment, and right leg problems. The patient's left-sided presentation extends to obesity and heel problems which may be contributing to or a consequence of the mentioned musculoskeletal complaints.

---


Patient ID: Neuro_00026
Age: 20
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a complex array of symptoms, encompassing various regions of t

In [161]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00049
Age: 20
Gender: Male

Symptom Diagnosis:
The patient presents with a constellation of symptoms localized to various anatomical regions indicative of musculoskeletal and potential neurological concerns. Specifically, the patient reports left-neck issues, right-neck discomfort, and right-sided back headache pain. Additionally, the patient experiences trouble with the left shoulder and right shoulder problems. Complaints also include intrascapular issues, pain in the right lateral elbow, left arm discomfort, problems with the left wrist, and complaints related to the right armband. The symptoms show a range of areas affected that may suggest multi-level involvement of the cervical and possibly upper thoracic spine.

---


Patient ID: Neuro_00009
Age: 39
Gender: Female

Symptom Diagnosis:
The pa

In [162]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00088
Age: 52
Gender: Male

Summary of Symptom Diagnosis:
The 52-year-old male patient presents with a complex array of musculoskeletal complaints involving multiple anatomical regions. Specifically, he reports left neck problems and pain in the right post-tibial area, accompanied by shoulder impingement and problems in both the left and right shoulders. The patient also describes dysfunction and discomfort in both hands, medial elbow problems on the left, and lumbago. In addition, he suffers from left post-tibial area discomfort, right hip arthritis, pain in the left anterior knee, right shin trouble, difficulties with the left footstool, and right hamstring issues. These symptoms suggest involvement of various musculoskeletal structures on both sides of the body.

---


Patient ID: Neuro_00073 


In [163]:
example = ''

chosen_idx = g==g_chosen[3]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00077
Age: 29
Gender: Female

Symptom Diagnosis:
The patient presents with a variety of musculoskeletal complaints including lumbago, issues with both right and left legs such as right shin pain, left rear thigh pain, and troubles with both ankles. The patient specifically reports problems with the left ankle, right ankle troubles, and a tendency for right Achilles issues. Besides these localized symptoms, the patient also describes conditions involving both Achilles tendons, with left Achilles tendon pain and right Achilles issues, and states the presence of left-sided obesity which may be contributing to the musculoskeletal concerns.

---


Patient ID: Neuro_00004
Age: 25
Gender: Male

Symptom Diagnosis:
The patient presented with right side ptosis (R pta), lower back pain (lumbago), pain in the

In [187]:
input_factors = '''  
0.
Based on the consideration above and the symptoms provided, the following factors seem to be the most relevant for grouping the patients:
For Group 'R shoulder impingement' = 0:
A specific factor is challenging to pinpoint due to the heterogeneity of symptoms. Thus, no specific DLI or radiculopathy fits all cases based solely on the provided summaries and the constraints of the task.
For Group 'R shoulder impingement' = 1:
"R Radiculopathy C5"
"R Radiculopathy C6"

"R Radiculopathy C5"
"R Radiculopathy C6"

"R Radiculopathy C5"
"R Radiculopathy C6"

1.
For the group with 'R shoulder impingement' = 0:
No specific factors have been identified due to the varied nature of the symptoms.
For the group with 'R shoulder impingement' = 1:
"R Radiculopathy C4-C5": Both patients present with right shoulder impingement and other symptoms that could be attributed to issues in the C4-C5 dermatomal regions.


"R Radiculopathy C5-C6": This is suggested due to the presentation of right shoulder impingement, which typically implicates nerve root compression or irritation at the C5-C6 levels.
"R Radiculopathy C6": In addition to shoulder impingement, the involvement of the thumb (which is innervated by C6) suggests a radiculopathy at this level.
"R Radiculopathy C7": This could be implicated by the symptoms in the lower arm and lateral elbow pain, which are areas typically innervated by the C7 nerve root.
"R Radiculopathy C8": Given the involvement of the thumbs and the mention of lower arm issues, as well as potential difficulties with the "thumbs up" action, this could suggest C8 involvement.

For Group with 'R shoulder impingement' = 1, the factors are:
"R Radiculopathy C5"
"R Radiculopathy C6"


2.
For group with 'R shoulder impingement' = 1 (since 'R shoulder impingement' is the defining feature of this group, the factors are related to the right shoulder):
"R Radiculopathy C4-C5": Patients in this group report issues that could be associated with nerve root compression or irritation in the right C4-C5 dermatome, which corresponds to the shoulder area where impingement is reported.
"R Radiculopathy C6-C8": Some patients report hand and wrist issues, which can be associated with these cervical dermatome areas.
For group with 'R shoulder impingement' = 0:
No specific radiculopathy or DLI patterns can be clearly derived from the available data for this group. The symptoms are widespread and do not consistently map to specific dermatome levels that would indicate a radiculopathy or DLI.

For 'R shoulder impingement' = 1 group (presence of specific right shoulder impingement):
"R Radiculopathy C4-C6"
This factor reflects the potential nerve root issues that could be contributing to the right shoulder impingement symptoms observed in this group.
For 'R shoulder impingement' = 0 group (absence of specific right shoulder impingement):
No specific factors can be determined from the provided data without further information on symptoms aligning with a DLI or radiculopathy within the cervical, thoracic, lumbar, or sacral dermatomes.

"R Radiculopathy C4" - This could explain the shoulder area issues and impingement.
"R Radiculopathy C5" - This might be related to the lateral upper arm pain and the ability to give a thumbs-up, suggesting that the C5 nerve root may be involved in the impingement and associated symptoms.


3.
"R Radiculopathy C4-C5"

For group with 'R shoulder impingement' = 1: "R Radiculopathy C5-C6" (since these dermatomes cover the shoulder area and lateral upper arm to the elbow, which are likely to be affected in cases of right shoulder impingement).

For the group with 'R shoulder impingement' = 0 (no specific right shoulder impingement):
DLI L1-L5
DLI S1-S5
For the group with 'R shoulder impingement' = 1 (specific right shoulder impingement):
R Radiculopathy C4-C5


'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['DLI L4-L5',
 'R C8 Radiculopathy',
 'R C7 Radiculopathy',
 'DLI L1-L2',
 'DLI S1-S2']

### confirmation

In [188]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

DLI C5-C6
R C4 Radiculopathy
R C6 Radiculopathy
R C7 Radiculopathy
L C6 Radiculopathy
L C5 Radiculopathy
DLI C4-C5
R C5 Radiculopathy
False


In [189]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 8: 100%|██████████| 9/9 [00:00<00:00, 732.32it/s]

[0.36319974 0.46921774 0.4166811  0.4356054  0.38644277 0.44301279
 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="R C6 Radiculopathy"];
4 [label=X5];
4 [label="R C7 Radiculopathy"];
5 [label=X6];
5 [label="L C6 Radiculopathy"];
6 [label=X7];
6 [label="L C5 Radiculopathy"];
7 [label=X8];
7 [label="DLI C4-C5"];
8 [label=X9];
8 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
7 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 3  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
6 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
6 -> 8  [arrowhead=odot, arrowtail=odot, 

In [190]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'R C7 Radiculopathy', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	5	3
0.8333333333333334	0.9487179487179487	0.7142857142857143	0.625


## iteration 4

### feedback

In [191]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.5982695885852573 21 [0.71428571 0.28571429]
1 0.5890026049647302 29 [0.72413793 0.27586207]
2 0.6931471805599453 12 [0.5 0.5]
3 0.0 38 [1.]
choose g==[0, 1, 2]


In [192]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00060
Age: 37
Gender: Male

Summary of Symptom Diagnosis:
The patient presents with pain and potential functional impairment localized to the left neck and both shoulders, encompassing the intra-scapular region. These symptoms suggest involvement of the cervical spine or surrounding musculoskeletal structures, possibly compromising nerve function, joint integrity, muscular or ligamentous tissue, or a combination thereof.

---


Patient ID: Neuro_00071
Age: 44
Gender: Male

Symptom Diagnosis:
The patient presents with a constellation of symptoms that include problems with the left neck, right shoulder, and right ham coupled with right shoulder trouble with specific difficulty in performing a 'thumbs up.' The patient also reports trouble in the left hip joint, lower left leg, and left ankle, in addi

In [193]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00042
Age: 47
Gender: Male

Symptom Diagnosis:
The patient presents with a constellation of symptoms ranging from neck pain to unilateral auditory perception issues, specifically right-sided tinnitus. He experiences headaches, issues with the left jaw, and has trouble with both shoulders—left shoulder impingement, and undefined issues with the right shoulder. There is a report of intrascapular problems contributing to discomfort. In terms of limb function, the patient has indicated that there is an issue with the left arm, without further details provided on the precise nature of these symptoms. However, the patient appears to have intact motor function as indicated by the report of the ability to give a 'thumbs up' with both hands.

---


Patient ID: Neuro_00011
Age: 56
Gender: Male

Summary of S

In [194]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---



---


Patient ID: Neuro_00044
Age: 33
Gender: Female

Summary of Symptom Diagnosis:
The patient has presented with pain localized to the neck and has been experiencing issues with both shoulders. Specifically, there is documentation of problems in the left shoulder as well as trouble in the right shoulder. These symptoms necessitate further evaluation to determine the underlying cause and appropriate interventions.

---


Patient ID: Neuro_00014
Age: 20
Gender: Male

Symptom Diagnosis Summary:
The patient presents with pain predominantly in the cervical and lumbosacral regions, manifested as neck pain, with specific complaints in the right neck area. Additionally, the patient reports issues with the right front axle, which could be descriptive of bodily or structural discomfort in that region. Musculoskeletal trou

In [195]:
input_factors = '''  
0.
Group with 'R shoulder impingement' = 0:
Patient Neuro_00060: Symptoms suggest issues with the left neck and both shoulders, which could be consistent with a "DLI C3-C4" due to the involvement of the shoulder area.
Patient Neuro_00071: Presents with right shoulder trouble, which could indicate a "R Radiculopathy C4-C5", but since the patient is categorized as not having R shoulder impingement, no specific radiculopathy associated with R shoulder impingement is indicated.
Patient Neuro_00061: Bilateral shoulder issues and right neck pain could suggest a "DLI C4-C5"; however, his symptoms are more severe on the left side, not specific to R shoulder impingement.
Group with 'R shoulder impingement' = 1:
Patient Neuro_00070: Reports impingement in the right shoulder which could indicate a "R Radiculopathy C5-C6", as these dermatome levels could affect the arm and elbow, areas where the patient has trouble.
Patient Neuro_00096: Presents with bilateral shoulder impingement and pain in right neck which could suggest a "R Radiculopathy C4-C5" or "R Radiculopathy C5-C6" given the symptomatic areas.
Patient Neuro_00054: Has right shoulder impingement, right lateral elbow pain, and complaints that could be consistent with "R Radiculopathy C5-C6" and "R Radiculopathy C6-C7" due to the involvement of the forearm and the thumb side of the hand.


"DLI C3-C4"
"R Radiculopathy C4-C5"
"DLI C4-C5"
"R Radiculopathy C5-C6"
For the group with 'R shoulder impingement' = 1:
"R Radiculopathy C4-C5"
"R Radiculopathy C6-C7"
"R Radiculopathy C5-C6"

DLI C4-C6: Suggested by the presence of bilateral shoulder issues and neck pain in the group without specified right shoulder impingement.
R Radiculopathy C4-C6: Suggested by the explicit mention of right shoulder impingement in the second group, which may be indicative of issues with nerve roots innervating the right shoulder area.


1.
Based on my analysis, the factors that appear to contribute to the allocation of patients into groups are as follows:
DLI C4-C5: This may be a potential factor as this location is relevant to shoulder issues, which are mentioned across multiple patients, though not specifically tied to 'R shoulder impingement'.
R Radiculopathy C5-C6: This factor is implicated in right shoulder impingement, which is the defining characteristic of group 1, and is supported by symptoms such as difficulties with thumb movements and other issues in the arms.
L Radiculopathy C5-C6: Although less clear due to less specific left-sided symptoms, this factor may play a role given the presence of symptoms such as left shoulder impingement and arm issues in patients of the group without R shoulder impingement.
In conclusion, the factors for each group may be:
For the group with 'R shoulder impingement' = 0: DLI C4-C5 and L Radiculopathy C5-C6
For the group with 'R shoulder impingement' = 1: R Radiculopathy C5-C6


Group with 'R shoulder impingement' = 0: No specific factors related to right shoulder impingement can be abstracted from the provided symptoms, as none of the patients have a clear diagnosis of right shoulder impingement. The symptoms mentioned do not allow for a definitive mapping to a specific dermatome location for DLI or radiculopathy.
Group with 'R shoulder impingement' = 1: Given the explicit mentions of right shoulder impingement and related symptoms, the following factors can be proposed:
"R Radiculopathy C5-C6": This factor is chosen because the symptoms of right shoulder impingement could involve the C5 and C6 nerve roots, which innervate the shoulder area and upper arm.
Additional factors might be considered if more detailed information on symptoms were provided to map onto specific dermatome regions for each patient. However, based on the current information, "R Radiculopathy C5-C6" is the most appropriate factor to abstract.


For Group 0 (without 'R shoulder impingement'):
"DLI C4-C5" or "L Radiculopathy C5": Due to left shoulder impingement and undefined issues with the right shoulder.
"L Radiculopathy C6": Due to issues with the left arm and differential motor ability in the left hand.
For Group 1 (with 'R shoulder impingement'):
"R Radiculopathy C5-C6": Right shoulder impingement and difficulty with 'thumbs up' gesture suggest involvement of these dermatomes.
Additional factors for individual patients with left-sided symptoms may be considered, but they do not contribute to the grouping based on 'R shoulder impingement'.



2.
For Group with 'R shoulder impingement' = 1:
"R Radiculopathy C4-C6": This is because the right shoulder impingement is closely associated with the C4-C6 dermatomes, which correspond to the shoulder area.
For Group with 'R shoulder impingement' = 0:
The provided summaries for this group do not clearly indicate radiculopathy or discoligamentous injury in a specific dermatomal region. They mention neck pain and shoulder issues, but without the clear indication of right shoulder impingement or other specific dermatomal symptoms, it's challenging to assign a specific factor based on the provided templates. More detailed information would be required to propose a specific factor for this group.

For patients in the group with 'R shoulder impingement' = 0:
No specific factors related to right shoulder impingement can be determined from the provided data since the group lacks this symptom.
For patients in the group with 'R shoulder impingement' = 1:
"R Radiculopathy C5" - This factor is chosen because the C5 dermatome covers the shoulder area where impingement is reported.
"R Radiculopathy C6" - Given that the C6 dermatome includes the thumb side of the hand and lateral forearm, issues in this area could also relate to the shoulder impingement symptoms.

"R Radiculopathy C5": This factor is chosen because the C5 dermatome covers the lateral upper arm to the elbow, which is an area that could be affected by right shoulder impingement.
"R Radiculopathy C6": The C6 dermatome affects the forearm and the thumb side of the hand. Given the proximity to the shoulder, disturbances here could be related to right shoulder impingement symptoms.
"R Radiculopathy C4": Although less common, C4 involvement could theoretically contribute to shoulder area issues, including impingement.

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['L C4 Radiculopathy', 'DLI C3-C4']

### confirmation

In [196]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

DLI C5-C6
R C4 Radiculopathy
R C6 Radiculopathy
R C7 Radiculopathy
DLI C3-C4
L C6 Radiculopathy
L C5 Radiculopathy
DLI C4-C5
R C5 Radiculopathy
False


In [197]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 9: 100%|██████████| 10/10 [00:00<00:00, 497.33it/s]

[0.36319974 0.46921774 0.4166811  0.4356054  0.38644277 0.4
 0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="R C6 Radiculopathy"];
4 [label=X5];
4 [label="R C7 Radiculopathy"];
5 [label=X6];
5 [label="DLI C3-C4"];
6 [label=X7];
6 [label="L C6 Radiculopathy"];
7 [label=X8];
7 [label="L C5 Radiculopathy"];
8 [label=X9];
8 [label="DLI C4-C5"];
9 [label=X10];
9 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
8 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 3  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];

In [198]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'R C7 Radiculopathy', 'DLI C3-C4', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	6	3
1.0	0.9615384615384616	0.8	0.6666666666666666


## iteration 5

### feedback

In [199]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.10656595882801999 45 [0.97777778 0.02222222]
1 0.5929533174474745 25 [0.72 0.28]
2 0.6474466390346325 20 [0.65 0.35]
3 0.6931471805599453 10 [0.5 0.5]
choose g==[0, 1, 2, 3]


In [200]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00033
Age: 37
Gender: Female

Symptom Diagnosis Summary:
The patient presents with lumbago characterized by lower back pain, as well as pain in the right frontal knee and trouble with the right ankle. These symptoms are suggestive of musculoskeletal issues but may also be indicative of nerve root involvement given the nature and distribution of the pains reported. 

---


Patient ID: Neuro_00055
Age: 56
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a constellation of symptoms including pain in the left pectoral area (L pta), pain at the front of the left knee, and unspecified trouble with the left knee and left footstool, suggesting potential mobility issues or structural disturbances. Additionally, the patient experiences a disorder within the medial aspect of the right knee 

In [202]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00020
Age: 53
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a complex constellation of symptoms that includes irritable bowel syndrome (IBS) and musculoskeletal complaints affecting various regions. Notably, there are problems reported with the left neck and right shoulder, as well as within the right intrascapular area. Additionally, the patient experiences difficulties with the left thumb ('thumbs up' sign), left wrist, and right hand. There is also discomfort in the left medial elbow, right lower back (lumbago), with associated obesity, and pain in the right rear thigh.

---


Patient ID: Neuro_00009
Age: 39
Gender: Female

Symptom Diagnosis:
The patient presents with a complex symptomatology including neck pain localized on the right side, issues with both shoulders, troub

In [203]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00035
Age: 55
Gender: Female

Symptom Diagnosis Summary:
The patient presents with a complex set of symptoms including neck pain, discomfort in the right ear, headaches, and a sensation of pharyngeal discomfort. Additionally, the patient is experiencing central chest disorders on the left side, bilateral shoulder problems, as well as trouble more specifically in the left shoulder. She also reports discomfort in the left arm and in the right bend of the arm. These symptoms suggest involvement of multiple anatomical structures and potential neural pathways.

---


Patient id: Neuro_00068
Age: 42
Gender: Male

Symptom diagnosis:
The patient presents with a range of musculoskeletal symptoms including left neck problems, complications with both shoulders, and additional trouble in the left and right ha

In [204]:
example = ''

chosen_idx = g==g_chosen[3]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00082
Age: 57
Gender: Female

Symptom Diagnosis Summary:
The patient presents with unilateral neck discomfort on the right side, associated with impingement and trouble in the left shoulder, as well as unspecified problems in the right shoulder, contributing to difficulties in upper limb function. There is also a reported discomfort in the left upper arm and problems affecting the left hand. The patient experiences lumbago, with symptoms reported on both sides, although the narrative suggests a predominant right-sided lumbago. The hip pain mentioned is bilateral with a specified arthritis in the right hip, while the left side appears to be less defined in characteristic. Additionally, sciatica symptoms are indicated exclusively on the left side (referred to as ischias), along with pain in the left

In [205]:
input_factors = '''  
0.
For the group with 'R shoulder impingement' = 0:
"DLI L4-L5" and "DLI S1" could be candidate factors for the first patient based on lower back pain and right ankle trouble.
"L Radiculopathy L4-S1" might be a factor for the second patient due to left knee pain and foot problems.
"DLI L1-L5" and "DLI S1" could be relevant for the third patient due to lower back pain, hip issues, and Achilles problems.
For the group with 'R shoulder impingement' = 1:
"R Radiculopathy C5-C6" is a likely factor due to the right shoulder impingement and lateral elbow pain.
"DLI L4-L5" could also be a factor given the presence of lumbago and right lower extremity issues.


"R Radiculopathy C4-C5": This is based on the patient's presentation of right shoulder impingement, which could be associated with radiculopathy at the C4-C5 dermatomes due to their innervation of the shoulder area.

For the group with 'R shoulder impingement' = 0 (no right shoulder impingement), no specific dermatome-related factors are provided that could be associated with discoligamentous injuries or radiculopathy specifically excluding right shoulder impingement.
For the group with 'R shoulder impingement' = 1 (presence of right shoulder impingement), the likely factor contributing to this allocation is:
"R Radiculopathy C5" or "R Radiculopathy C6"

1.
For the group with 'R shoulder impingement' = 1, the factors might be:
"R Radiculopathy C5"
"R Radiculopathy C6"

Group with 'R shoulder impingement' = 0:
There are no specific factors indicating right shoulder impingement relating to dermatomes in this group.
Group with 'R shoulder impingement' = 1:
"DLI C4-C5": This could be related to the right shoulder impingement as these dermatomes cover the shoulder area.
"R Radiculopathy C5": Since the C5 dermatome can affect the shoulder and lateral upper arm, right-sided radiculopathy at C5 could contribute to right shoulder impingement symptoms.
"R Radiculopathy C6": Given the involvement of the thumb and upper limb, right C6 radiculopathy may also play a role in the constellation of symptoms that includes right shoulder impingement.

"R Radiculopathy C5" due to the right shoulder impingement symptoms.
"R Radiculopathy C6" due to the involvement of the thumb (right thumbs up discomfort).

2.
"R Radiculopathy C5": This is based on the common symptom of right shoulder impingement, which is often associated with C5 nerve root issues as the C5 dermatome includes the shoulder area.
"R Radiculopathy C6": Considering that the C6 dermatome covers the thumb side of the hand and the forearm, and given that some patients report symptoms in these areas, it may also be involved.


For the group with 'R shoulder impingement' = 0:
No specific factors related to right shoulder impingement and dermatomes have been identified based on the provided summaries.
For the group with 'R shoulder impingement' = 1:
"R Radiculopathy C5": Since C5 dermatome is related to the shoulder area and impingement symptoms are reported in the right shoulder.
"R Radiculopathy C6": As C6 can affect the thumb side of the hand and forearm, which may be related to the impingement symptoms in the right shoulder.

For the group with 'R shoulder impingement' = 1, the factors are:
"R Radiculopathy C5-C6"


3.
"R Radiculopathy C4-C6"

"R Radiculopathy C5-C6"

R Radiculopathy C4-C6"

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['L S1 Radiculopathy', 'L L4 Radiculopathy']

### confirmation

In [206]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

DLI C5-C6
R C4 Radiculopathy
R C6 Radiculopathy
R C7 Radiculopathy
DLI C3-C4
L C6 Radiculopathy
L C5 Radiculopathy
DLI C4-C5
R C5 Radiculopathy
True


In [207]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 9: 100%|██████████| 10/10 [00:00<00:00, 713.77it/s]

[0.36319974 0.46921774 0.4166811  0.4356054  0.38644277 0.4
 0.44301279 0.34220843 0.40665944 0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="DLI C5-C6"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="R C6 Radiculopathy"];
4 [label=X5];
4 [label="R C7 Radiculopathy"];
5 [label=X6];
5 [label="DLI C3-C4"];
6 [label=X7];
6 [label="L C6 Radiculopathy"];
7 [label=X8];
7 [label="L C5 Radiculopathy"];
8 [label=X9];
8 [label="DLI C4-C5"];
9 [label=X10];
9 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
8 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 3  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];

In [208]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['DLI C5-C6', 'R C4 Radiculopathy', 'R C6 Radiculopathy', 'R C7 Radiculopathy', 'DLI C3-C4', 'L C6 Radiculopathy', 'L C5 Radiculopathy', 'DLI C4-C5', 'R C5 Radiculopathy']
3	6	3
1.0	0.9615384615384616	0.8	0.6666666666666666


##  GPT-4-Turbo Zeroshot

https://poe.com/s/WC7hgur7zFFSeOuaXSTq

In [143]:
input_factors = '''  
DLI C4-C5
DLI C5-C6
R Radiculopathy C4
R Radiculopathy C5
R Radiculopathy C6
R Radiculopathy C7
R Radiculopathy C8
R Radiculopathy T1

DLI C4-C5
DLI C5-C6
R Radiculopathy C5
R Radiculopathy C6

"R Radiculopathy C5"
"R Radiculopathy C6"
"DLI C4-C5"
"DLI C5-C6"
'''.split('\n')
proposed_zero_shot_factors = get_variables_with_data(input_factors)
proposed_zero_shot_factors

['R T11 Radiculopathy',
 'R T1 Radiculopathy',
 'R T12 Radiculopathy',
 'R T10 Radiculopathy',
 'R C6 Radiculopathy',
 'R C7 Radiculopathy',
 'R C8 Radiculopathy',
 'DLI C4-C5',
 'R C4 Radiculopathy',
 'DLI C5-C6',
 'R C5 Radiculopathy']

In [144]:
print(*Eval_Factor_Discovery(proposed_zero_shot_factors), sep='\t')
print(*Eval_Factor_Confirmation(proposed_zero_shot_factors, proposed_zero_shot_factors, mode = 'ancestors'), sep='\t')

3	5	6
0.8333333333333334	0.9102564102564102	0.588235294117647	0.45454545454545453


# Mistral-Medium

https://poe.com/Mistral-Medium


https://poe.com/s/1yZr77sxWIjBvWQjVRdC

In [28]:
V = set(['R shoulder impingement'])
all_factors = set(['R shoulder impingement'])
factors = [ f for f in V if f!= target_name]
factors

[]

## iteration 1

In [29]:
print(propose_prompt_with_data_in_front(init_data))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data

 

### Group with 'R shoulder impingement'=0

----


Patient ID: Neuro_00083
Age: 10
Gender: Female

Symptom Diagnosis:
The patient presents with a multiplicity of musculoskeletal complaints including left neck problems, leading to discomfort. Additionally, there is an indication of left shoulder impingement, accompanied by non-specific problems with the right shoulder. The patient also reports issues in the lower body, including concerns with the left adductor tendon, unspecified lumbago, discomfort in the left hip joint, and pain in the right shin. Further symptoms involve trouble with the left knee, pain in the left hamstring, issues with the left toe, and pain located in the left rear thigh.

----


Patient ID: Neuro_00040
Age: 19
Gender: Male

Symptom Diagnosis:
The patient presents with a range of symptoms predominantly relating to the lower back and left lowe

In [30]:

input_factors = '''  
R Radiculopathy C4
R Radiculopathy C5

Template 2: R Radiculopathy C4
Template 2: R Radiculopathy C5
Template 2: R Radiculopathy C6

R Radiculopathy C4
R Radiculopathy C5


'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['R C6 Radiculopathy', 'R C4 Radiculopathy', 'R C5 Radiculopathy']

### confirmation

In [31]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

R C5 Radiculopathy
R C4 Radiculopathy
R C6 Radiculopathy
False


In [32]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 3: 100%|██████████| 4/4 [00:00<00:00, 1089.43it/s]

[0.36319974 0.4356054  0.4166811  0.39444071]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="R C6 Radiculopathy"];
2 [label=X3];
2 [label="R C4 Radiculopathy"];
3 [label=X4];
3 [label="R C5 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
}



In [34]:
# meta_data
print(*Eval_Factor_Discovery(factors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, factors, mode = 'ancestors'), sep='\t')

3	3	0
0.5	0.9615384615384616	0.6666666666666666	1.0


In [35]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['R C6 Radiculopathy', 'R C4 Radiculopathy', 'R C5 Radiculopathy']
3	3	0
0.5	0.9615384615384616	0.6666666666666666	1.0


## iteration 2

### feedback

In [36]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.5763341277567499 19 [0.73684211 0.26315789]
1 0.410116318288409 56 [0.85714286 0.14285714]
2 0.5929533174474745 25 [0.72 0.28]
choose g==[0, 1, 2]


In [37]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00093
Age: 47
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a range of musculoskeletal complaints, including left neck problems and right partial thromboplastin time (R pta) issues. There is a noted left shoulder impingement alongside general trouble in both the left and right shoulders. The patient also reports intracapsular joint problems without specifying the joint. Lower extremity issues are described, including right leg problems, left partial thromboplastin time (L pta) issues, right hip arthritis, and pain located at the left frontal knee. Additional symptoms include right shin discomfort, left tåledbesvär (assumed to relate to problems with the left ankle or toe), and issues with the right hamstring (R ham).

---


Patient ID: Neuro_00038

---


Patient ID: Neuro_0008

In [38]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---



---


Patient ID: Neuro_00010
Age: 12
Gender: Female

Symptom Diagnosis Summary:
The patient presents with a constellation of symptoms, including pharyngeal discomfort and difficulty with the right jaw, potentially indicating an oropharyngeal or temporomandibular condition. Additionally, she experiences right adductor tendonitis and right hip arthritis, suggestive of musculoskeletal issues in the corresponding regions. There is also a complaint of discomfort in the left hamstring (L ham), which could be musculotendinous in origin. The patient's symptoms are further complicated by left-sided obesity, which may contribute to her musculoskeletal discomfort. The right hamstring (R ham) is also noted as a site of discomfort for the patient.

---


Patient ID: Neuro_00013
Age: 57
Gender: Male

Symptom Diagnosis Summary:

In [39]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00088
Age: 52
Gender: Male

Summary of Symptom Diagnosis:
The 52-year-old male patient presents with a complex array of musculoskeletal complaints involving multiple anatomical regions. Specifically, he reports left neck problems and pain in the right post-tibial area, accompanied by shoulder impingement and problems in both the left and right shoulders. The patient also describes dysfunction and discomfort in both hands, medial elbow problems on the left, and lumbago. In addition, he suffers from left post-tibial area discomfort, right hip arthritis, pain in the left anterior knee, right shin trouble, difficulties with the left footstool, and right hamstring issues. These symptoms suggest involvement of various musculoskeletal structures on both sides of the body.

---


Patient ID: Neuro_00046
A

In [40]:
input_factors = '''  
0.

R Radiculopathy C4
L Radiculopathy C4
L Radiculopathy C6

R Radiculopathy C4-C5 dermatome (Right shoulder impingement)

R Radiculopathy C3-C4
R Radiculopathy C4-C5
R Radiculopathy C5-C6

1.

Right Radiculopathy C4-C5

R Radiculopathy C4
R Radiculopathy C5

R Radiculopathy C4: This factor relates to symptoms in the shoulder area, which is consistent with right shoulder impingement.
R Radiculopathy C5: This factor pertains to symptoms in the lateral upper arm to the elbow, which may be associated with right shoulder impingement.
R Radiculopathy C6: This factor concerns symptoms in the forearm and the thumb side of the hand, which can potentially be related to right shoulder impingement.



2.
R Radiculopathy C4: This factor relates to symptoms in the shoulder area, which is consistent with right shoulder impingement.
R Radiculopathy C5: This factor pertains to symptoms in the lateral upper arm to the elbow, which may be associated with right shoulder impingement.
R Radiculopathy C6: This factor concerns symptoms in the forearm and the thumb side of the hand, which can potentially be related to right shoulder impingement.

"R Radiculopathy C4"
"R Radiculopathy C5"
"R Radiculopathy C6"

Right Radiculopathy C4-C5: This factor corresponds to right shoulder impingement and is present in all patients in Group 1.


'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['R C3 Radiculopathy', 'L C4 Radiculopathy', 'L C6 Radiculopathy']

### confirmation

In [41]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

R C5 Radiculopathy
R C6 Radiculopathy
L C6 Radiculopathy
R C4 Radiculopathy
False


In [42]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 4: 100%|██████████| 5/5 [00:00<00:00, 789.32it/s]

[0.36319974 0.4356054  0.44301279 0.39444071 0.4166811 ]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="R C6 Radiculopathy"];
2 [label=X3];
2 [label="L C6 Radiculopathy"];
3 [label=X4];
3 [label="R C5 Radiculopathy"];
4 [label=X5];
4 [label="R C4 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 2  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [43]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['R C6 Radiculopathy', 'L C6 Radiculopathy', 'R C5 Radiculopathy', 'R C4 Radiculopathy']
3	3	1
0.5	0.9487179487179487	0.6000000000000001	0.75


## iteration 3

### feedback

In [44]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.4293230219306162 13 [0.84615385 0.15384615]
1 0.362210557135449 51 [0.88235294 0.11764706]
2 0.6637254974722313 29 [0.62068966 0.37931034]
3 0.410116318288409 7 [0.85714286 0.14285714]
choose g==[0, 1, 2, 3]


In [45]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00061
Age: 29
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a complex array of symptoms which include pain localized to the right neck and bilateral shoulder issues characterized by impingement and generalized trouble, more so on the left side. Additionally, the patient reports experiencing difficulties with the right thumb (associated with a positive "thumbs up" sign), left hand troubles suggestive of functional impairment, and right leg problems. The patient's left-sided presentation extends to obesity and heel problems which may be contributing to or a consequence of the mentioned musculoskeletal complaints.

---


Patient ID: Neuro_00090
Age: 12
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a spectrum of musculoskeletal complaints involving multiple ar

In [46]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00047, an 11-year-old male, presents with a constellation of symptoms including IBS (irritable bowel syndrome), lumbago (lower back pain), various right leg issues, localized obesity on the left side, discomfort in the right hamstring, problems associated with the right heel, symptoms of right achillodynia (pain in the Achilles tendon), and pain in the back area associated with the breast (presumably pectoral or upper back region). These symptoms collectively suggest a complex presentation affecting multiple body systems and areas.

---


Clinical Notes:
Patient ID: Neuro_00089
Age: 58
Gender: Female

Symptom Diagnosis Summary:
The patient presents with a variety of symptoms, including right (R) neck discomfort, left (L) ear issues, and persistent R tinnitus. There are also reports of R eye compli

In [47]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00048
Age: 52
Gender: Female

Symptom Diagnosis Summary:
The patient presents with a multisystemic range of symptoms including irritable bowel syndrome (IBS) and various musculoskeletal complaints. She is experiencing issues with her left shoulder, described as an impingement and associated trouble, accompanied by interscapular complaints localized to the left side. Extending from these concerns, the patient reports problems affecting her left arm and left medial elbow. Additionally, similar symptoms are noted on the right medial elbow. Lower limb-related difficulties are reported as well, with the patient describing trouble with the left shin and both left and right ankles. On the right side, there is mention of hamstring pain and tear issues, while on the left, the patient exhibits signs of achi

In [48]:
example = ''

chosen_idx = g==g_chosen[3]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00049
Age: 20
Gender: Male

Symptom Diagnosis:
The patient presents with a constellation of symptoms localized to various anatomical regions indicative of musculoskeletal and potential neurological concerns. Specifically, the patient reports left-neck issues, right-neck discomfort, and right-sided back headache pain. Additionally, the patient experiences trouble with the left shoulder and right shoulder problems. Complaints also include intrascapular issues, pain in the right lateral elbow, left arm discomfort, problems with the left wrist, and complaints related to the right armband. The symptoms show a range of areas affected that may suggest multi-level involvement of the cervical and possibly upper thoracic spine.

---


Patient ID: Neuro_00094
Age: 49
Gender: Female

Symptom diagnosis summary

In [49]:
input_factors = '''  
0.

"R Radiculopathy C4-C5"
"DLI C4-C5"

R Radiculopathy C3-C8
R Radiculopathy T7-T12 (optional, depending on the significance of lumbago in patient Neuro_00054)

R Radiculopathy C3-C8
R Radiculopathy T7-T12 (optional, depending on the significance of lumbago in patient Neuro_00054)


1.

R Radi C4-C5

R Radiculopathy C4
R Radiculopathy C5


R Radiculopathy C4-C5 (Right shoulder impingement)
L Radiculopathy C4-C5 (Left shoulder impingement or unspecified problems)
L Radiculopathy L1-L5 (Lumbago and associated symptoms)


2.
For the group with right shoulder impingement (group 1), the factor is "R Radiculopathy C4" or "DLI C4-R".

Factor 1: R Radiculopathy C4-C5
Factor 2: L/R Radiculopathy C1-C3
Factor 3: L/R Radiculopathy C6-C8

R Radiculopathy C4: This factor represents potential nerve root involvement causing right shoulder impingement symptoms.
R Radiculopathy C5: This factor represents potential nerve root involvement causing symptoms affecting the lateral upper arm and elbow, which are often associated with shoulder impingement.



3.
"R Radiculopathy C4": This factor represents the primary difference between the two groups, which is the presence of right shoulder impingement in one group.
"L Radiculopathy C4" or "L Radiculopathy C5": These factors account for the left shoulder problems reported by the patient in the group with right shoulder impingement.
"Bilateral Radiculopathy C6": This factor represents the difficulties with both thumbs experienced by the patient in the group with right shoulder impingement.
"Bilateral Radiculopathy C5" or "Bilateral Radiculopathy C6": These factors account for the bilateral lateral elbow pain reported by the patient in the group with right shoulder impingement.
"R Radiculopathy C6": This factor represents the right lower arm issues reported by the patient in the group with right shoulder impingement.
"L Radiculopathy L5" or "L Radiculopathy S1": These factors account for the left ankle trouble reported by the patient in the group with right shoulder impingement, although this symptom is not directly related to the shoulder impingement.


L Radiculopathy C4
L Radiculopathy C5
L Radiculopathy C6
L Radiculopathy C7
L Radiculopathy C8
R Radiculopathy C4
R Radiculopathy C5
R Radiculopathy C6
R Radiculopathy C7
R Radiculopathy C8
Group with 'R shoulder impingement' = 1:

R Radiculopathy C4
R Radiculopathy C5
L Radiculopathy C4
L Radiculopathy C5
L Radiculopathy C6
L Radiculopathy C7

R Radiculopathy C5-C6

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['L L1 Radiculopathy',
 'L L5 Radiculopathy',
 'L C5 Radiculopathy',
 'R L5 Radiculopathy',
 'DLI C4-C5',
 'L C8 Radiculopathy',
 'R T7 Radiculopathy',
 'R C7 Radiculopathy',
 'R T10 Radiculopathy',
 'L C3 Radiculopathy',
 'R S1 Radiculopathy',
 'L S1 Radiculopathy',
 'R C8 Radiculopathy',
 'L C7 Radiculopathy']

### confirmation

In [50]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

R C6 Radiculopathy
L C5 Radiculopathy
R C4 Radiculopathy
DLI C4-C5
R C7 Radiculopathy
R C5 Radiculopathy
L C6 Radiculopathy
L C7 Radiculopathy
False


In [51]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 8: 100%|██████████| 9/9 [00:00<00:00, 521.33it/s]

[0.36319974 0.4356054  0.39444071 0.34220843 0.44301279 0.40270162
 0.4166811  0.40665944 0.38644277]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="R C6 Radiculopathy"];
2 [label=X3];
2 [label="R C5 Radiculopathy"];
3 [label=X4];
3 [label="L C5 Radiculopathy"];
4 [label=X5];
4 [label="L C6 Radiculopathy"];
5 [label=X6];
5 [label="L C7 Radiculopathy"];
6 [label=X7];
6 [label="R C4 Radiculopathy"];
7 [label=X8];
7 [label="DLI C4-C5"];
8 [label=X9];
8 [label="R C7 Radiculopathy"];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 3  [arrowhead=normal, arrowtail=normal, dir=both];
6 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
7 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 3  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
7 -> 3  [arrowhead=normal, arrowtail=odot, dir=both];
5 -> 8  [arrowhead=odot, a

In [52]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['R C6 Radiculopathy', 'R C5 Radiculopathy', 'L C6 Radiculopathy', 'R C4 Radiculopathy', 'DLI C4-C5']
3	4	1
0.6666666666666666	0.9615384615384616	0.7272727272727273	0.8


## iteration 4

### feedback

In [53]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.5982695885852573 21 [0.71428571 0.28571429]
1 0.6853142072764582 16 [0.5625 0.4375]
2 0.6108643020548935 20 [0.7 0.3]
3 0.11045304667259806 43 [0.97674419 0.02325581]
choose g==[0, 1, 2, 3]


In [54]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00019
Age: 57
Gender: Female

Symptom Diagnosis:
The patient presents with a constellation of symptoms involving bilateral shoulders with impingement syndrome primarily on the left shoulder, left neck problems, generalized left arm issues, and left shoulder trouble. Furthermore, the patient experiences discomfort in the lower back region acknowledged as left lumbago, right hip issues diagnosed as arthritis, problems associated with the right hamstring (referred to as right ham), trouble with the left ankle, and issues pertaining to the left toe.

---


Patient ID: Neuro_00065
Age: 26
Gender: Male

Symptom Diagnosis Summary:
The patient presents with an array of musculoskeletal symptoms affecting multiple areas of the body. The patient reports experiencing problems specifically with the left neck a

In [55]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient id: Neuro_00068
Age: 42
Gender: Male

Symptom diagnosis:
The patient presents with a range of musculoskeletal symptoms including left neck problems, complications with both shoulders, and additional trouble in the left and right hands. The patient describes specific difficulties with the left small finger, alongside issues with both medial elbows. These varied symptoms are suggestive of a multifocal neurological or musculoskeletal condition affecting multiple limbs and warrant thorough investigation.

---


Patient ID: Neuro_00014
Age: 20
Gender: Male

Symptom Diagnosis Summary:
The patient presents with pain predominantly in the cervical and lumbosacral regions, manifested as neck pain, with specific complaints in the right neck area. Additionally, the patient reports issues with the right front axle, whic

In [56]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00099
Age: 28
Gender: Female

Symptom Diagnosis Summary:
The patient presented with a range of symptoms affecting various bilateral anatomical regions. She reports having trouble with the right (R) jaw and right shoulder, which could be indicative of joint or musculoskeletal issues in these areas. Additionally, she is experiencing left (L) lateral elbow pain and left wrist problems, both of which could be related to overuse injuries or strain. Furthermore, she describes discomfort related to the left adductor tendon, suggesting a possible tendinopathy or strain. The patient also reports lumbago, a general term for lower back pain, which may result from a variety of underlying conditions. Obesity on the right side presents an atypical clinical picture that requires further investigation. She is als

In [57]:
example = ''

chosen_idx = g==g_chosen[3]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00041
Age: 43
Gender: Male

Symptom Diagnosis:
The patient presents with lumbago characterized by lower back pain, as well as radiating pain to the left lower extremity (L pta), pain in the left hip joint, and trouble with both the left and right ankles. Additionally, the patient has a concern regarding right-sided obesity which may be contributing to the musculoskeletal symptoms.

---


Patient ID: Neuro_00079
Age: 40
Gender: Male

Symptom Diagnosis:
The patient presents with a host of musculoskeletal complaints including left neck problems, a consistent headache, issues with the left jaw, and left shoulder impingement characterized by trouble in the left shoulder. Additionally, the patient reports problems with the right shoulder, lumbago, lateral obesity, and pain in both the left and right rea

In [58]:
input_factors = '''  
0.

R Radiculopathy C4
R Radiculopathy C5
R Radiculopathy C6


L Radiculopathy C4
L Radiculopathy C5
L Radiculopathy C6


R Radiculopathy C4
L Radiculopathy C4



1.
R Radiculopathy C4  
DLI C5-C6
R Radiculopathy T2 

R Radiculopathy C4-C5

R Radiculopathy C4-C5
DLI C4-C5



2.
R Radiculopathy C4
R Radiculopathy C5-C6
L Radiculopathy C4-C6



R Radiculopathy C4-C5 (based on right shoulder impingement symptoms)
R Radiculopathy C5-C6 (based on right shoulder impingement and "thumbs up" difficulty symptoms)


R Radiculopathy C4-C6 (Right shoulder impingement dermatome)
L Radiculopathy C4-C6 (Left shoulder impingement dermatome for Group 1 patients with right shoulder impingement)
L Radiculopathy C6-C8 (Left upper extremity dermatome for both groups)
R Radiculopathy C6-C8 (Right upper extremity dermatome for Group 1 patients)
L Radiculopathy L2-L5 (Lower limb dermatome for Group 2 patients)

3.

L Radiculopathy C5-C6
L Radiculopathy L4-L5
L Radiculopathy L5-S1

R Radiculopathy C4-C5


DLI C4
R Radiculopathy C5





'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['L L4 Radiculopathy', 'DLI C5-C6']

### confirmation

In [59]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

R C6 Radiculopathy
L C5 Radiculopathy
R C4 Radiculopathy
DLI C4-C5
R C7 Radiculopathy
DLI C5-C6
R C5 Radiculopathy
L C6 Radiculopathy
L C7 Radiculopathy
False


In [60]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 9: 100%|██████████| 10/10 [00:00<00:00, 655.06it/s]

[0.36319974 0.4356054  0.39444071 0.34220843 0.44301279 0.40270162
 0.4166811  0.46921774 0.40665944 0.38644277]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="R C6 Radiculopathy"];
2 [label=X3];
2 [label="R C5 Radiculopathy"];
3 [label=X4];
3 [label="L C5 Radiculopathy"];
4 [label=X5];
4 [label="L C6 Radiculopathy"];
5 [label=X6];
5 [label="L C7 Radiculopathy"];
6 [label=X7];
6 [label="R C4 Radiculopathy"];
7 [label=X8];
7 [label="DLI C5-C6"];
8 [label=X9];
8 [label="DLI C4-C5"];
9 [label=X10];
9 [label="R C7 Radiculopathy"];
6 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
7 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
8 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 3  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 8  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 8  [arrowhead=odot, arrowtail=od

In [61]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['R C6 Radiculopathy', 'R C5 Radiculopathy', 'L C5 Radiculopathy', 'L C6 Radiculopathy', 'R C4 Radiculopathy', 'DLI C5-C6', 'DLI C4-C5']
3	5	2
0.8333333333333334	0.9615384615384616	0.7692307692307694	0.7142857142857143


## iteration 5

### feedback

In [62]:
focus_factor = 'R shoulder impingement'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(V) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = [set_g[idx] for idx, this_entropy_value in enumerate(entropy_values) if this_entropy_value>0]
print(f'choose g=={g_chosen}')


0 0.5982695885852573 21 [0.71428571 0.28571429]
1 0.5890026049647302 29 [0.72413793 0.27586207]
2 0.6931471805599453 12 [0.5 0.5]
3 0.0 38 [1.]
choose g==[0, 1, 2]


In [63]:
example = ''

chosen_idx = g==g_chosen[0]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00026
Age: 20
Gender: Male

Symptom Diagnosis Summary:
The patient presents with a complex array of symptoms, encompassing various regions of the musculoskeletal system. He reports pain in the right neck, both shoulders (with more specific trouble in the right shoulder), and within the intrascapular region. Alongside these upper-body complaints, the patient experiences lumbago, bilateral thigh problems with noted pain on the left side, and posterior thigh pain on the left. Lower extremity issues include right hip arthritis, right ankle trouble, as well as problems with the left lateral foot and right heel. The patient also mentions upper abdominal discomfort and has been identified as having left-sided obesity.

---


Patient ID: Neuro_00038

---


Patient ID: Neuro_00090
Age: 12
Gender: Male

Sym

In [64]:
example = ''

chosen_idx = g==g_chosen[1]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient ID: Neuro_00035
Age: 55
Gender: Female

Symptom Diagnosis Summary:
The patient presents with a complex set of symptoms including neck pain, discomfort in the right ear, headaches, and a sensation of pharyngeal discomfort. Additionally, the patient is experiencing central chest disorders on the left side, bilateral shoulder problems, as well as trouble more specifically in the left shoulder. She also reports discomfort in the left arm and in the right bend of the arm. These symptoms suggest involvement of multiple anatomical structures and potential neural pathways.

---


Patient ID: Neuro_00009
Age: 39
Gender: Female

Symptom Diagnosis:
The patient presents with a complex symptomatology including neck pain localized on the right side, issues with both shoulders, trouble with the right elbow, and problems w

In [65]:
example = ''

chosen_idx = g==g_chosen[2]
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,0]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'R shoulder impingement' = 0

---


Patient id: Neuro_00068
Age: 42
Gender: Male

Symptom diagnosis:
The patient presents with a range of musculoskeletal symptoms including left neck problems, complications with both shoulders, and additional trouble in the left and right hands. The patient describes specific difficulties with the left small finger, alongside issues with both medial elbows. These varied symptoms are suggestive of a multifocal neurological or musculoskeletal condition affecting multiple limbs and warrant thorough investigation.

---


Patient ID: Neuro_00066
Age: 19
Gender: Male

Symptoms Diagnosis Summary:
The 19-year-old male patient presents with pain in the neck region and has reported experiencing issues with his left shoulder. The patient describes these issues as troubles and problems with the left shoulder, but exact nature or

In [66]:
input_factors = '''  
0.
DLI C3-C4
DLI C5-C6
R Radiculopathy C4
R Radiculopathy C5

R Radiculopathy C5
R Radiculopathy C6

R Radiculopathy C4
R Radiculopathy C5
DLI C4-C5 


1.
R Radiculopathy C4-C5 
L Radiculopathy C4-C5 
R Radiculopathy C1-C4 
DLI C1-C7 

R Radiculopathy C5
L Radiculopathy C6/C7/C8


R Radiculopathy C5

2.
Right C4 radiculopathy (R C4 radiculopathy)
Right C5 radiculopathy (R C5 radiculopathy)


R Radiculopathy C4: Right shoulder area involvement.
R Radiculopathy C5: Right lateral upper arm involvement.
R Radiculopathy C6: Right forearm and thumb-side hand involvement.
R Radiculopathy C7: Right forearm and middle-finger involvement.


R Radiculopathy C5
R Radiculopathy C6
L Radiculopathy C3-C5

'''.split('\n')

proposed_factors = get_variables_with_data(input_factors)
proposed_factors = [f for f in proposed_factors if f in data_interface.columns]
new_factors = [f for f in proposed_factors if  (f not in all_factors)]
new_factors = list(np.array(new_factors)[data_interface[new_factors].values.std(0) > 0])

for f in new_factors:
    all_factors.add(f)

new_factors

['DLI C1-C2', 'DLI C3-C4', 'DLI C6-C7']

### confirmation

In [67]:
annotated_name = [target_name]+[f for f in all_factors if f != target_name]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

V_prev = deepcopy(V)
V = set(['R shoulder impingement'])
V_idx = []

candidate_factors = np.array([f for f in annotated_name if f not in V])

for _ in range(len(annotated_name)):
    batch_names = np.array([f for idx, f in enumerate(annotated_name) if f not in V])
    batch_idx = np.array([idx for idx, f in enumerate(annotated_name) if f not in V])
    batch_p_values = np.array([ci_test(0,idx,V_idx) for idx, f in enumerate(annotated_name) if f not in V])
    if len(batch_p_values)==0:
        break
    if batch_p_values.min() >= alpha:
        break
    for p_value, idx, f in zip(batch_p_values, batch_idx, batch_names):
        if p_value < alpha:
            V_idx.append(int(idx))
            V.add(f)
            print(f)
print(V_prev==V)

R C6 Radiculopathy
L C5 Radiculopathy
R C4 Radiculopathy
DLI C4-C5
R C7 Radiculopathy
DLI C5-C6
DLI C3-C4
R C5 Radiculopathy
L C6 Radiculopathy
L C7 Radiculopathy
False


In [68]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 10: 100%|██████████| 11/11 [00:00<00:00, 433.76it/s]

[0.36319974 0.4356054  0.39444071 0.34220843 0.44301279 0.40270162
 0.4166811  0.46921774 0.40665944 0.38644277 0.4       ]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label="R shoulder impingement"];
1 [label=X2];
1 [label="R C6 Radiculopathy"];
2 [label=X3];
2 [label="R C5 Radiculopathy"];
3 [label=X4];
3 [label="L C5 Radiculopathy"];
4 [label=X5];
4 [label="L C6 Radiculopathy"];
5 [label=X6];
5 [label="L C7 Radiculopathy"];
6 [label=X7];
6 [label="R C4 Radiculopathy"];
7 [label=X8];
7 [label="DLI C5-C6"];
8 [label=X9];
8 [label="DLI C4-C5"];
9 [label=X10];
9 [label="R C7 Radiculopathy"];
10 [label=X11];
10 [label="DLI C3-C4"];
6 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
7 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
8 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 7  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 3  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 8  [arrowhead=odot, arrowtail=odo

In [69]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
print(*Eval_Factor_Discovery(possible_ancestors), sep='\t')
print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')

['R C6 Radiculopathy', 'R C5 Radiculopathy', 'L C5 Radiculopathy', 'L C6 Radiculopathy', 'R C4 Radiculopathy', 'DLI C5-C6', 'DLI C4-C5', 'DLI C3-C4']
3	6	2
1.0	0.9743589743589743	0.8571428571428572	0.75
